In [ ]:
from importlib import reload
#reload(Utilities)
# NOTE: To reload a class imported as, e.g., 
# from module import class
# One must call:
#   1. import module
#   2. reload module
#   3. from module import class

import sys, os
import re
from pathlib import Path
import json
import pickle

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version
import copy

import itertools

import pyodbc
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
import matplotlib.colors as mcolors
import matplotlib.cm as cm #e.g. for cmap=cm.jet
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from MECPODf import MECPODf
from MECPOAn import MECPOAn
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
#sys.path.insert(0, os.path.join(os.path.realpath('..'), 'Utilities'))
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_General
import Plot_Box_sns
import Plot_Hist
import Plot_Bar
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
run_testing_data=True
fig_num=0

dovs_and_end_events_data_dir = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data'
if run_testing_data:
    dovs_and_end_events_data_dir = os.path.join(dovs_and_end_events_data_dir, r'TESTING_DATASETS')

files_dir_outg             = os.path.join(dovs_and_end_events_data_dir, r'EndEvents')
files_dir_outg_prim_strict = os.path.join(dovs_and_end_events_data_dir, r'EndEvents_prim_strict')
files_dir_no_outg = os.path.join(dovs_and_end_events_data_dir, r'EndEvents_NoOutg')

file_path_glob = r'end_events_[0-9]*.csv'
file_path_regex = None

assert_all_cols_equal=False #Seems new EndEvents have curr_acct_cls_cd as well...
include_normalize_by_nSNs=True
inclue_zero_counts=True
return_multiindex_outg_reason=False
return_normalized_separately=False

days_min_outg_td_window=1
days_max_outg_td_window=30

if include_normalize_by_nSNs and not return_normalized_separately:
    normalize_by_nSNs_included=True
else:
    normalize_by_nSNs_included=False
    
xfmr_equip_typ_nms_of_interest = ['TRANSFORMER, OH', 'TRANSFORMER, UG']

In [ ]:
save_dfs_to_pkl = False
read_dfs_from_pkl = False
save_dir_base_pkls = os.path.join(Utilities.get_local_data_dir(), r'dovs_and_end_events_data', 'rcpo_dfs')

assert(save_dfs_to_pkl+read_dfs_from_pkl <=1) # Should never both read and write!

In [ ]:
#-------------------------
outg_save_subdir = f'outg_td_window_{days_min_outg_td_window}_to_{days_max_outg_td_window}_days'
save_dir_base_outg_pkls = os.path.join(save_dir_base_pkls, outg_save_subdir)
if save_dfs_to_pkl and not os.path.exists(save_dir_base_outg_pkls):
    os.makedirs(save_dir_base_outg_pkls)
#-------------------------

# TODO!~
Should all : be replaced with \: (except. e.g., ?:)
Also, do all . need to be replaced with \. when I mean period instead of any character?!

In [ ]:
patterns_to_replace=[
    r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', 
   (
       (
           r'(Under Voltage)\s*'\
           r'([0-9a-zA-Z]*)?\s*'\
           r'(\([0-9a-zA-Z\s]*\))\s*'\
           r'([0-9a-zA-Z]*)?\s?'\
           r'(for meter\:?\s*)'\
           r'(?:(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+)?[\s:,.]*'\
           r'(?:Phase\s{1,2}[ABC](?:(?:\s*and\s*[ABC])|(?:,\s*[ABC])*))?\s*'\
           r'(Voltage out of tolerance)?'
       ), 
       'AMIEndEvents.under_voltage_match_func'
   ), 
   (
       (
           r'(Last Gasp\s*-\s*[0-9a-zA-Z\s]*)[\s\:,.]*'\
           r'.*'\
           r'(Fail Reason: .*)$'
       ), 
       'AMIEndEvents.last_gasp_reduce_func'
   ), 
    (r'(Angle out of tolerance) \[.*\]', r'\1'), 
    (r'(NIC Power Restore Trap Received from device).*', r'\1'), #TODO This has much info, like last gasp
    (
        (
            r'(Requested operation .* could not be applied to the given device type and firmware version.) '\
            r'Device, DeviceType: .*, Firmware Version: .*$'
        ), 
        r'\1'
    ), 
    ('meterN/A', 'meter'),
    (r'(Meter needs explicit time sync.) Drift: -?\d* s, (Encountered Problems:\s*.*), Meter_Time', r'\1 \2'), 
    (r'(Meter Program Seal mismatch for Device) \[Device ID, MAC Id\] = .*', r'\1'), 
    (r'Device Time: .* Failed Device Reason: (.*) Reboot Counter: .* Refresh Counter: .*', r'\1'), 
    (r'(Ignoring (?:Interval|Register) Read data for device as it has time in the future) .*', r'\1'), 
    (r'(Secure association operation failed consecutively for 1 times for [0-9a-zA-Z]{4}.) .*', r'\1'), 
    (r'Device, (Last Gasp State: .*), (Detector State: .*), Reboot Count: \d*', r'\1, \2'), 
    (r'(Detected end of voltage sag on meter).*', r'\1'), 
    (r'(Detected end of voltage swell on meter).*', r'\1'), 
    r'N/A', 
    (r'\s{2,}', ' ')
]

In [ ]:
if not read_dfs_from_pkl:
    start = time.time()
    rcpo_df_OG, ede_typeid_to_reason_df_OG = OutageMdlrPrep.build_rcpx_df_from_EndEvents_in_csvs(    
        files_dir=files_dir_outg, 
        mp_df=None, 
        file_path_glob=file_path_glob, 
        file_path_regex=file_path_regex, 
        patterns_to_replace=patterns_to_replace, 
        min_outg_td_window=datetime.timedelta(days=days_min_outg_td_window),
        max_outg_td_window=datetime.timedelta(days=days_max_outg_td_window),
        build_ede_typeid_to_reason_df=True, 
        batch_size=1000, 
        cols_and_types_to_convert_dict=None, 
        to_numeric_errors='coerce', 
        assert_all_cols_equal=assert_all_cols_equal, 
        include_normalize_by_nSNs=include_normalize_by_nSNs, 
        inclue_zero_counts=inclue_zero_counts, 
        return_multiindex_outg_reason=return_multiindex_outg_reason, 
        return_normalized_separately=return_normalized_separately, 
        verbose=True, 
        n_update=1, 
        is_no_outage=False, 
        include_prem_nbs=True
    )
    print(time.time()-start)
    #-------------------------
    if save_dfs_to_pkl:
        rcpo_df_OG.to_pickle(os.path.join(save_dir_base_outg_pkls, 'rcpo_df_OG.pkl'))
        ede_typeid_to_reason_df_OG.to_pickle(os.path.join(save_dir_base_outg_pkls, 'ede_typeid_to_reason_df_OG.pkl'))        
else:
    rcpo_df_OG = pd.read_pickle(os.path.join(save_dir_base_outg_pkls, 'rcpo_df_OG.pkl'))
    ede_typeid_to_reason_df_OG = pd.read_pickle(os.path.join(save_dir_base_outg_pkls, 'ede_typeid_to_reason_df_OG.pkl'))    

In [ ]:
print(rcpo_df_OG.shape)

In [ ]:
if not read_dfs_from_pkl:
    start = time.time() 
    rcpo_prim_strict_df_OG, ede_typeid_to_reason_prim_strict_df_OG = OutageMdlrPrep.build_rcpx_df_from_EndEvents_in_csvs(    
        files_dir=files_dir_outg_prim_strict, 
        mp_df=None, 
        file_path_glob=file_path_glob, 
        file_path_regex=file_path_regex, 
        patterns_to_replace=patterns_to_replace, 
        min_outg_td_window=datetime.timedelta(days=days_min_outg_td_window),
        max_outg_td_window=datetime.timedelta(days=days_max_outg_td_window),
        build_ede_typeid_to_reason_df=True, 
        batch_size=50, 
        cols_and_types_to_convert_dict=None, 
        to_numeric_errors='coerce', 
        assert_all_cols_equal=assert_all_cols_equal, 
        include_normalize_by_nSNs=include_normalize_by_nSNs, 
        inclue_zero_counts=inclue_zero_counts, 
        return_multiindex_outg_reason=return_multiindex_outg_reason, 
        return_normalized_separately=return_normalized_separately, 
        verbose=True, 
        n_update=1, 
        is_no_outage=False, 
        include_prem_nbs=True
    )
    print(time.time()-start)
    #-------------------------
    if save_dfs_to_pkl:
        rcpo_prim_strict_df_OG.to_pickle(os.path.join(save_dir_base_outg_pkls, 'rcpo_prim_strict_df_OG.pkl'))
        ede_typeid_to_reason_prim_strict_df_OG.to_pickle(os.path.join(save_dir_base_outg_pkls, 'ede_typeid_to_reason_prim_strict_df_OG.pkl'))        
else:
    rcpo_prim_strict_df_OG = pd.read_pickle(os.path.join(save_dir_base_outg_pkls, 'rcpo_prim_strict_df_OG.pkl'))
    ede_typeid_to_reason_prim_strict_df_OG = pd.read_pickle(os.path.join(save_dir_base_outg_pkls, 'ede_typeid_to_reason_prim_strict_df_OG.pkl'))    

In [ ]:
print(rcpo_prim_strict_df_OG.shape)

In [ ]:
if not read_dfs_from_pkl:
    start = time.time()
    rcpo_no_outg_df_OG, ede_typeid_to_reason_no_outg_df_OG = OutageMdlrPrep.build_rcpx_df_from_EndEvents_in_csvs(    
        files_dir=files_dir_no_outg, 
        mp_df=None, 
        file_path_glob=file_path_glob, 
        file_path_regex=file_path_regex, 
        patterns_to_replace=patterns_to_replace, 
        min_outg_td_window=datetime.timedelta(days=days_min_outg_td_window),
        max_outg_td_window=datetime.timedelta(days=days_max_outg_td_window),
        build_ede_typeid_to_reason_df=True, 
        batch_size=1000, 
        cols_and_types_to_convert_dict=None, 
        to_numeric_errors='coerce', 
        assert_all_cols_equal=assert_all_cols_equal, 
        include_normalize_by_nSNs=include_normalize_by_nSNs, 
        inclue_zero_counts=inclue_zero_counts, 
        return_multiindex_outg_reason=return_multiindex_outg_reason, 
        return_normalized_separately=return_normalized_separately, 
        verbose=True, 
        n_update=1, 
        outg_rec_nb_col='trsf_pole_nb', 
        is_no_outage=True, 
        include_prem_nbs=True
    )
    print(time.time()-start)
    #-------------------------
    if save_dfs_to_pkl:
        rcpo_no_outg_df_OG.to_pickle(os.path.join(save_dir_base_outg_pkls, 'rcpo_no_outg_df_OG.pkl'))
        ede_typeid_to_reason_no_outg_df_OG.to_pickle(os.path.join(save_dir_base_outg_pkls, 'ede_typeid_to_reason_no_outg_df_OG.pkl'))        
else:
    rcpo_no_outg_df_OG = pd.read_pickle(os.path.join(save_dir_base_outg_pkls, 'rcpo_no_outg_df_OG.pkl'))
    ede_typeid_to_reason_no_outg_df_OG = pd.read_pickle(os.path.join(save_dir_base_outg_pkls, 'ede_typeid_to_reason_no_outg_df_OG.pkl'))    

In [ ]:
print(rcpo_no_outg_df_OG.shape)

In [ ]:
ede_typeid_to_reason_df = AMIEndEvents.combine_two_ede_typeid_to_reason_dfs(
    ede_typeid_to_reason_df_OG, 
    ede_typeid_to_reason_prim_strict_df_OG, 
    sort=True
)
ede_typeid_to_reason_df = AMIEndEvents.combine_two_ede_typeid_to_reason_dfs(
    ede_typeid_to_reason_df, 
    ede_typeid_to_reason_no_outg_df_OG, 
    sort=True
)

In [ ]:
print(ede_typeid_to_reason_df_OG.shape)
print(ede_typeid_to_reason_prim_strict_df_OG.shape)
print(ede_typeid_to_reason_no_outg_df_OG.shape)

In [ ]:
print(len(set(ede_typeid_to_reason_df_OG.index.tolist() + 
              ede_typeid_to_reason_prim_strict_df_OG.index.tolist() + 
              ede_typeid_to_reason_no_outg_df_OG.index.tolist())))
print(ede_typeid_to_reason_df.shape)

In [ ]:
reason_to_ede_typeid_df = AMIEndEvents.invert_ede_typeid_to_reason_df(ede_typeid_to_reason_df)

In [ ]:
reason_to_ede_typeid_df

In [ ]:
# reason_counts_df = AMIEndEvents.get_net_reason_counts_for_outages_from_rcpo_df(
#     rcpo_df = rcpo_df, 
#     reason_col='reason', 
#     normalize_by_nSNs_included=normalize_by_nSNs_included
# )
# # reason_counts_df = reason_counts_df.sort_values(by=('counts_norm', 'mean'), ascending=False)
# # ax = reason_counts_df.plot.bar(y=('counts_norm', 'mean'), rot=90)

In [ ]:
# reason_counts_prim_strict_df = AMIEndEvents.get_net_reason_counts_for_outages_from_rcpo_df(
#     rcpo_df = rcpo_prim_strict_df, 
#     reason_col='reason', 
#     normalize_by_nSNs_included=normalize_by_nSNs_included
# )
# # reason_counts_prim_strict_df = reason_counts_prim_strict_df.sort_values(by=('counts_norm', 'mean'), ascending=False)
# # ax = reason_counts_prim_strict_df.plot.bar(y=('counts_norm', 'mean'), rot=90)

In [ ]:
rcpo_df = rcpo_df_OG.copy()
rcpo_prim_strict_df = rcpo_prim_strict_df_OG.copy()
rcpo_no_outg_df = rcpo_no_outg_df_OG.copy()

In [ ]:
if not read_dfs_from_pkl:
    start=time.time()
    #-------------------------
    rcpo_df_raw  = MECPODf.project_level_0_columns_from_rcpo_wide(rcpo_df, 'counts', droplevel=True)
    #-----
    rcpo_df_raw = MECPODf.add_outage_SNs_to_rcpo_df(
        rcpo_df=rcpo_df_raw, 
        set_outage_nSNs=True, 
        include_outage_premise_nbs=True    
    )
    #-----
    rcpo_df_raw = MECPODf.add_prim_SNs_to_rcpo_df(
        rcpo_df=rcpo_df_raw, 
        direct_SNs_in_outgs_df=None, 
        outg_rec_nb_col='index', 
        prim_SNs_col='direct_serial_numbers', 
        set_prim_nSNs=True, 
        sort_SNs=True, 
        build_direct_SNs_in_outgs_df_kwargs={}
    )
    #-------------------------
    rcpo_df_norm = MECPODf.project_level_0_columns_from_rcpo_wide(rcpo_df, 'counts_norm', droplevel=True)
    rcpo_df_norm_by_outg_nSNs = MECPODf.build_rcpo_df_norm_by_outg_nSNs(rcpo_df_raw)
    rcpo_df_norm_by_prim_nSNs = MECPODf.build_rcpo_df_norm_by_prim_nSNs(
        rcpo_df_raw=rcpo_df_raw, 
        direct_SNs_in_outgs_df=None, 
        outg_rec_nb_col='index', 
        prim_nSNs_col='_prim_nSNs', 
        prim_SNs_col='_prim_SNs', 
        other_SNs_col_tags_to_ignore=['_SNs', '_nSNs', '_prem_nbs', '_nprem_nbs'], 
        drop_prim_nSNs_eq_0=True, 
        new_level_0_val='counts_norm_by_prim_nSNs', 
        remove_SNs_cols=False, 
        build_direct_SNs_in_outgs_df_kwargs=dict(equip_typ_nms_of_interest=xfmr_equip_typ_nms_of_interest)
    )
    #-------------------------
    icpo_df_raw = MECPODf.convert_rcpo_to_icpo_df(
        rcpo_df=rcpo_df_raw, 
        reason_to_ede_typeid_df=reason_to_ede_typeid_df, 
        is_norm=False
    )

    icpo_df_norm = MECPODf.convert_rcpo_to_icpo_df(
        rcpo_df=rcpo_df_norm, 
        reason_to_ede_typeid_df=reason_to_ede_typeid_df, 
        is_norm=True, 
        counts_col='_nSNs'
    )

    icpo_df_norm_by_outg_nSNs = MECPODf.convert_rcpo_to_icpo_df(
        rcpo_df=rcpo_df_norm_by_outg_nSNs, 
        reason_to_ede_typeid_df=reason_to_ede_typeid_df, 
        is_norm=True, 
        counts_col='_outg_nSNs'
    )

    icpo_df_norm_by_prim_nSNs = MECPODf.convert_rcpo_to_icpo_df(
        rcpo_df=rcpo_df_norm_by_prim_nSNs, 
        reason_to_ede_typeid_df=reason_to_ede_typeid_df, 
        is_norm=True, 
        counts_col='_prim_nSNs'
    )
    #-------------------------
    print(time.time()-start)
    #-------------------------
    if save_dfs_to_pkl:
        #-------------------------
        rcpo_df_raw.to_pickle(os.path.join(save_dir_base_outg_pkls, 'rcpo_df_raw.pkl'))
        rcpo_df_norm.to_pickle(os.path.join(save_dir_base_outg_pkls, 'rcpo_df_norm.pkl'))
        rcpo_df_norm_by_outg_nSNs.to_pickle(os.path.join(save_dir_base_outg_pkls, 'rcpo_df_norm_by_outg_nSNs.pkl'))
        rcpo_df_norm_by_prim_nSNs.to_pickle(os.path.join(save_dir_base_outg_pkls, 'rcpo_df_norm_by_prim_nSNs.pkl'))
        #-----
        icpo_df_raw.to_pickle(os.path.join(save_dir_base_outg_pkls, 'icpo_df_raw.pkl'))
        icpo_df_norm.to_pickle(os.path.join(save_dir_base_outg_pkls, 'icpo_df_norm.pkl'))
        icpo_df_norm_by_outg_nSNs.to_pickle(os.path.join(save_dir_base_outg_pkls, 'icpo_df_norm_by_outg_nSNs.pkl'))
        icpo_df_norm_by_prim_nSNs.to_pickle(os.path.join(save_dir_base_outg_pkls, 'icpo_df_norm_by_prim_nSNs.pkl'))
        #-------------------------
else:
    rcpo_df_raw = pd.read_pickle(os.path.join(save_dir_base_outg_pkls, 'rcpo_df_raw.pkl'))
    rcpo_df_norm = pd.read_pickle(os.path.join(save_dir_base_outg_pkls, 'rcpo_df_norm.pkl'))
    rcpo_df_norm_by_outg_nSNs = pd.read_pickle(os.path.join(save_dir_base_outg_pkls, 'rcpo_df_norm_by_outg_nSNs.pkl'))
    rcpo_df_norm_by_prim_nSNs = pd.read_pickle(os.path.join(save_dir_base_outg_pkls, 'rcpo_df_norm_by_prim_nSNs.pkl'))
    #-----
    icpo_df_raw = pd.read_pickle(os.path.join(save_dir_base_outg_pkls, 'icpo_df_raw.pkl'))
    icpo_df_norm = pd.read_pickle(os.path.join(save_dir_base_outg_pkls, 'icpo_df_norm.pkl'))
    icpo_df_norm_by_outg_nSNs = pd.read_pickle(os.path.join(save_dir_base_outg_pkls, 'icpo_df_norm_by_outg_nSNs.pkl'))
    icpo_df_norm_by_prim_nSNs = pd.read_pickle(os.path.join(save_dir_base_outg_pkls, 'icpo_df_norm_by_prim_nSNs.pkl'))

In [ ]:
if not read_dfs_from_pkl:
    start=time.time()
    #-------------------------
    rcpo_prim_strict_df_raw  = MECPODf.project_level_0_columns_from_rcpo_wide(rcpo_prim_strict_df, 'counts', droplevel=True)
    #-----
    rcpo_prim_strict_df_raw = MECPODf.add_outage_SNs_to_rcpo_df(
        rcpo_df=rcpo_prim_strict_df_raw, 
        set_outage_nSNs=True, 
        include_outage_premise_nbs=True    
    )
    #-----
    rcpo_prim_strict_df_raw = MECPODf.add_prim_SNs_to_rcpo_df(
        rcpo_df=rcpo_prim_strict_df_raw, 
        direct_SNs_in_outgs_df=None, 
        outg_rec_nb_col='index', 
        prim_SNs_col='direct_serial_numbers', 
        set_prim_nSNs=True, 
        sort_SNs=True, 
        build_direct_SNs_in_outgs_df_kwargs={}
    )
    #-------------------------
    rcpo_prim_strict_df_norm = MECPODf.project_level_0_columns_from_rcpo_wide(rcpo_prim_strict_df, 'counts_norm', droplevel=True)
    rcpo_prim_strict_df_norm_by_outg_nSNs = MECPODf.build_rcpo_df_norm_by_outg_nSNs(rcpo_prim_strict_df_raw)
    rcpo_prim_strict_df_norm_by_prim_nSNs = MECPODf.build_rcpo_df_norm_by_prim_nSNs(
        rcpo_df_raw=rcpo_prim_strict_df_raw, 
        direct_SNs_in_outgs_df=None, 
        outg_rec_nb_col='index', 
        prim_nSNs_col='_prim_nSNs', 
        prim_SNs_col='_prim_SNs', 
        other_SNs_col_tags_to_ignore=['_SNs', '_nSNs', '_prem_nbs', '_nprem_nbs'], 
        drop_prim_nSNs_eq_0=True, 
        new_level_0_val='counts_norm_by_prim_nSNs', 
        remove_SNs_cols=False, 
        build_direct_SNs_in_outgs_df_kwargs=dict(equip_typ_nms_of_interest=xfmr_equip_typ_nms_of_interest)
    )
    #-------------------------
    icpo_prim_strict_df_raw = MECPODf.convert_rcpo_to_icpo_df(
        rcpo_df=rcpo_prim_strict_df_raw, 
        reason_to_ede_typeid_df=reason_to_ede_typeid_df, 
        is_norm=False
    )

    icpo_prim_strict_df_norm = MECPODf.convert_rcpo_to_icpo_df(
        rcpo_df=rcpo_prim_strict_df_norm, 
        reason_to_ede_typeid_df=reason_to_ede_typeid_df, 
        is_norm=True, 
        counts_col='_nSNs'
    )

    icpo_prim_strict_df_norm_by_outg_nSNs = MECPODf.convert_rcpo_to_icpo_df(
        rcpo_df=rcpo_prim_strict_df_norm_by_outg_nSNs, 
        reason_to_ede_typeid_df=reason_to_ede_typeid_df, 
        is_norm=True, 
        counts_col='_outg_nSNs'
    )

    icpo_prim_strict_df_norm_by_prim_nSNs = MECPODf.convert_rcpo_to_icpo_df(
        rcpo_df=rcpo_prim_strict_df_norm_by_prim_nSNs, 
        reason_to_ede_typeid_df=reason_to_ede_typeid_df, 
        is_norm=True, 
        counts_col='_prim_nSNs'
    )
    #-------------------------
    print(time.time()-start)
    #-------------------------
    if save_dfs_to_pkl:
        #-------------------------
        rcpo_prim_strict_df_raw.to_pickle(os.path.join(save_dir_base_outg_pkls, 'rcpo_prim_strict_df_raw.pkl'))
        rcpo_prim_strict_df_norm.to_pickle(os.path.join(save_dir_base_outg_pkls, 'rcpo_prim_strict_df_norm.pkl'))
        rcpo_prim_strict_df_norm_by_outg_nSNs.to_pickle(os.path.join(save_dir_base_outg_pkls, 'rcpo_prim_strict_df_norm_by_outg_nSNs.pkl'))
        rcpo_prim_strict_df_norm_by_prim_nSNs.to_pickle(os.path.join(save_dir_base_outg_pkls, 'rcpo_prim_strict_df_norm_by_prim_nSNs.pkl'))
        #-----
        icpo_prim_strict_df_raw.to_pickle(os.path.join(save_dir_base_outg_pkls, 'icpo_prim_strict_df_raw.pkl'))
        icpo_prim_strict_df_norm.to_pickle(os.path.join(save_dir_base_outg_pkls, 'icpo_prim_strict_df_norm.pkl'))
        icpo_prim_strict_df_norm_by_outg_nSNs.to_pickle(os.path.join(save_dir_base_outg_pkls, 'icpo_prim_strict_df_norm_by_outg_nSNs.pkl'))
        icpo_prim_strict_df_norm_by_prim_nSNs.to_pickle(os.path.join(save_dir_base_outg_pkls, 'icpo_prim_strict_df_norm_by_prim_nSNs.pkl'))
        #-------------------------
else:
    rcpo_prim_strict_df_raw = pd.read_pickle(os.path.join(save_dir_base_outg_pkls, 'rcpo_prim_strict_df_raw.pkl'))
    rcpo_prim_strict_df_norm = pd.read_pickle(os.path.join(save_dir_base_outg_pkls, 'rcpo_prim_strict_df_norm.pkl'))
    rcpo_prim_strict_df_norm_by_outg_nSNs = pd.read_pickle(os.path.join(save_dir_base_outg_pkls, 'rcpo_prim_strict_df_norm_by_outg_nSNs.pkl'))
    rcpo_prim_strict_df_norm_by_prim_nSNs = pd.read_pickle(os.path.join(save_dir_base_outg_pkls, 'rcpo_prim_strict_df_norm_by_prim_nSNs.pkl'))
    #-----
    icpo_prim_strict_df_raw = pd.read_pickle(os.path.join(save_dir_base_outg_pkls, 'icpo_prim_strict_df_raw.pkl'))
    icpo_prim_strict_df_norm = pd.read_pickle(os.path.join(save_dir_base_outg_pkls, 'icpo_prim_strict_df_norm.pkl'))
    icpo_prim_strict_df_norm_by_outg_nSNs = pd.read_pickle(os.path.join(save_dir_base_outg_pkls, 'icpo_prim_strict_df_norm_by_outg_nSNs.pkl'))
    icpo_prim_strict_df_norm_by_prim_nSNs = pd.read_pickle(os.path.join(save_dir_base_outg_pkls, 'icpo_prim_strict_df_norm_by_prim_nSNs.pkl'))

In [ ]:
if not read_dfs_from_pkl:
    start=time.time()
    #-------------------------
    rcpo_no_outg_df_raw  = MECPODf.project_level_0_columns_from_rcpo_wide(rcpo_no_outg_df, 'counts', droplevel=True)
    rcpo_no_outg_df_norm = MECPODf.project_level_0_columns_from_rcpo_wide(rcpo_no_outg_df, 'counts_norm', droplevel=True)
    #-----
    #-------------------------
    icpo_no_outg_df_raw = MECPODf.convert_rcpo_to_icpo_df(
        rcpo_df=rcpo_no_outg_df_raw, 
        reason_to_ede_typeid_df=reason_to_ede_typeid_df, 
        is_norm=False
    )

    icpo_no_outg_df_norm = MECPODf.convert_rcpo_to_icpo_df(
        rcpo_df=rcpo_no_outg_df_norm, 
        reason_to_ede_typeid_df=reason_to_ede_typeid_df, 
        is_norm=True, 
        counts_col='_nSNs'
    )
    #-------------------------
    print(time.time()-start)
    #-------------------------
    if save_dfs_to_pkl:
        #-------------------------
        rcpo_no_outg_df_raw.to_pickle(os.path.join(save_dir_base_outg_pkls, 'rcpo_no_outg_df_raw.pkl'))
        rcpo_no_outg_df_norm.to_pickle(os.path.join(save_dir_base_outg_pkls, 'rcpo_no_outg_df_norm.pkl'))
        #-----
        icpo_no_outg_df_raw.to_pickle(os.path.join(save_dir_base_outg_pkls, 'icpo_no_outg_df_raw.pkl'))
        icpo_no_outg_df_norm.to_pickle(os.path.join(save_dir_base_outg_pkls, 'icpo_no_outg_df_norm.pkl'))
else:
    rcpo_no_outg_df_raw = pd.read_pickle(os.path.join(save_dir_base_outg_pkls, 'rcpo_no_outg_df_raw.pkl'))
    rcpo_no_outg_df_norm = pd.read_pickle(os.path.join(save_dir_base_outg_pkls, 'rcpo_no_outg_df_norm.pkl'))
    #-----
    icpo_no_outg_df_raw = pd.read_pickle(os.path.join(save_dir_base_outg_pkls, 'icpo_no_outg_df_raw.pkl'))
    icpo_no_outg_df_norm = pd.read_pickle(os.path.join(save_dir_base_outg_pkls, 'icpo_no_outg_df_norm.pkl'))

In [ ]:
# Remove SNs cols
#-------------------------
rcpo_df_norm                = MECPODf.remove_SNs_cols_from_rcpo_df(rcpo_df_norm)
rcpo_df_norm_by_outg_nSNs   = MECPODf.remove_SNs_cols_from_rcpo_df(rcpo_df_norm_by_outg_nSNs)
rcpo_df_norm_by_prim_nSNs   = MECPODf.remove_SNs_cols_from_rcpo_df(rcpo_df_norm_by_prim_nSNs)
#-------------------------
rcpo_prim_strict_df_norm                = MECPODf.remove_SNs_cols_from_rcpo_df(rcpo_prim_strict_df_norm)
rcpo_prim_strict_df_norm_by_outg_nSNs   = MECPODf.remove_SNs_cols_from_rcpo_df(rcpo_prim_strict_df_norm_by_outg_nSNs)
rcpo_prim_strict_df_norm_by_prim_nSNs   = MECPODf.remove_SNs_cols_from_rcpo_df(rcpo_prim_strict_df_norm_by_prim_nSNs)
#-------------------------
rcpo_no_outg_df_norm                = MECPODf.remove_SNs_cols_from_rcpo_df(rcpo_no_outg_df_norm)

In [ ]:
#--------------------------------------------------
# Make sure DFs have the same set of reasons, filling in empty entries where needed
# DO RAW SEPARATELY, SINCE THOSE HAVE SERIAL NUMBER COLUMNS!
#--------------------------------------------------
AMIEndEvents.make_reason_counts_per_outg_columns_equal_dfs_list(
    rcpo_dfs = [
        rcpo_df_raw, 
        rcpo_prim_strict_df_raw, 
        rcpo_no_outg_df_raw
    ], 
    same_order=True, 
    inplace=True
)
#-----
AMIEndEvents.make_reason_counts_per_outg_columns_equal_dfs_list(
    rcpo_dfs = [
        rcpo_df_norm, 
        rcpo_df_norm_by_outg_nSNs, 
        rcpo_df_norm_by_prim_nSNs, 

        rcpo_prim_strict_df_norm, 
        rcpo_prim_strict_df_norm_by_outg_nSNs, 
        rcpo_prim_strict_df_norm_by_prim_nSNs, 

        rcpo_no_outg_df_norm
    ], 
    same_order=True, 
    inplace=True
)

# BEGIN: NEW PLOTTING STUFF

In [ ]:
rcpo_df_norm.shape

In [ ]:
rcpo_df_norm['Under Voltage (CA000400) cleared for meter'].describe()

In [ ]:
rcpo_df_norm.describe()

In [ ]:
rcpo_df_norm[rcpo_df_norm.mean().sort_values(ascending=False).index.tolist()].describe()

In [ ]:
rcpo_df_norm[rcpo_df_norm.mean().sort_values(ascending=False).index.tolist()].describe().T

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
order=rcpo_df_norm.mean().sort_values(ascending=False).index.tolist()
rcpo_df_norm[order].describe().T.drop('count', axis=1).plot(ax=ax0)
ax0.set_xticks(range(len(order)))
ax0.set_xticklabels(order)
ax0.tick_params(axis='x', labelrotation=90, labelsize=7.0)

In [ ]:
idk = rcpo_df_norm[rcpo_df_norm.mean().sort_values(ascending=False).index.tolist()].describe().T

In [ ]:
idk

In [ ]:
idk35 = rcpo_df_norm[rcpo_df_norm.mean().sort_values(ascending=False).index.tolist()].quantile(0.35)

In [ ]:
rcpo_df_norm[rcpo_df_norm.mean().sort_values(ascending=False).index.tolist()].quantile(0.35)

In [ ]:
idk35.name='35%'

In [ ]:
idk.merge(idk35, left_index=True, right_index=True)

In [ ]:
quantiles = np.arange(0.1,1.0,0.1).tolist()
quantiles.extend([0.95,0.975,0.99,0.999])

In [ ]:
rcpo_df_norm.mean()

In [ ]:
rcpo_df_norm[rcpo_df_norm.mean().sort_values(ascending=False).index.tolist()]

In [ ]:
rcpo_df_norm[rcpo_df_norm.mean().sort_values(ascending=False).index.tolist()].astype(float).quantile(0.90)

In [ ]:
quant_series = []
for quant in quantiles:
    q_i = rcpo_df_norm[rcpo_df_norm.mean().sort_values(ascending=False).index.tolist()].astype(float).quantile(quant)
    q_i.name = f'{round(quant*100,1)}%'
    quant_series.append(q_i)

In [ ]:
quant_df = pd.concat(quant_series, axis=1)

In [ ]:
quant_df

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
order=rcpo_df_norm.mean().sort_values(ascending=False).index.tolist()
quant_df.loc[order].plot(ax=ax0)
ax0.set_xticks(range(len(order)))
ax0.set_xticklabels(order)
ax0.tick_params(axis='x', labelrotation=90, labelsize=7.0)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
order=rcpo_df_norm.mean().sort_values(ascending=False).index.tolist()[:10]
quant_df.loc[order].plot(ax=ax0)
ax0.set_xticks(range(len(order)))
ax0.set_xticklabels(order)
ax0.tick_params(axis='x', labelrotation=90, labelsize=7.0)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
sns.boxplot(ax=ax0, x="reason", y="value", data=pd.melt(rcpo_df_norm), 
            order=rcpo_df_norm.mean().sort_values(ascending=False).index.tolist())
ax0.tick_params(axis='x', labelrotation=90, labelsize=7.0)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
sns.boxplot(ax=ax0, x="reason", y="value", data=pd.melt(rcpo_df_norm), 
            order=rcpo_df_norm.mean().sort_values(ascending=False).index.tolist())
ax0.tick_params(axis='x', labelrotation=90, labelsize=7.0)
ax0.set_yscale('log')

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
sns.boxplot(ax=ax0, x="reason", y="value", data=pd.melt(rcpo_df_norm), 
            order=rcpo_df_norm.mean().sort_values(ascending=False).index.tolist())
ax0.tick_params(axis='x', labelrotation=90, labelsize=7.0)
ax0.set_ylim(ymin=0.0, ymax=15.0)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
sns.boxplot(ax=ax0, x="reason", y="value", data=pd.melt(rcpo_df_norm), showfliers=False, 
            order=rcpo_df_norm.mean().sort_values(ascending=False).index.tolist())
ax0.tick_params(axis='x', labelrotation=90, labelsize=7.0)

In [ ]:
rcpo_df_norm

In [ ]:
pd.melt(rcpo_df_norm)

# END: NEW PLOTTING STUFF

# BEGIN: STANDARD PLOTTING STUFF

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
sns.barplot(ax=ax0, data=rcpo_df_norm)
ax0.tick_params(axis='x', labelrotation=90, labelsize=7.0)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
sns.barplot(ax=ax0, data=rcpo_df_norm, ci='sd')
ax0.tick_params(axis='x', labelrotation=90, labelsize=7.0)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
sns.barplot(ax=ax0, data=rcpo_df_norm, order=rcpo_df_norm.mean().sort_values(ascending=False).index.tolist())
ax0.tick_params(axis='x', labelrotation=90, labelsize=7.0)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
sns.barplot(ax=ax0, data=rcpo_prim_strict_df_norm, order=rcpo_prim_strict_df_norm.mean().sort_values(ascending=False).index.tolist())
ax0.tick_params(axis='x', labelrotation=90, labelsize=7.0)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[6, 14])
sns.barplot(
    ax=ax0, 
    data=rcpo_prim_strict_df_norm, 
    order=rcpo_prim_strict_df_norm.mean().sort_values(ascending=False).index.tolist(), 
    orient='h')
ax0.tick_params(axis='x', labelrotation=90, labelsize=7.0)
print('width ', ax0.patches[0].get_width())
print('height ', ax0.patches[0].get_height())
print('x ', ax0.patches[0].get_x())
print('y ', ax0.patches[0].get_y())
print()
print('width ', ax0.patches[1].get_width())
print('height ', ax0.patches[1].get_height())
print('x ', ax0.patches[1].get_x())
print('y ', ax0.patches[1].get_y())
print()
print('width ', ax0.patches[2].get_width())
print('height ', ax0.patches[2].get_height())
print('x ', ax0.patches[2].get_x())
print('y ', ax0.patches[2].get_y())

In [ ]:
fig, ax1 = plt.subplots(1, 1, num=fig_num, figsize=[6, 14])
sns.barplot(
    ax=ax1, 
    data=rcpo_prim_strict_df_norm, 
    order=rcpo_prim_strict_df_norm.mean().sort_values(ascending=False).index.tolist(), 
    orient='v')
ax1.tick_params(axis='x', labelrotation=90, labelsize=7.0)
print('width ', ax1.patches[0].get_width())
print('height ', ax1.patches[0].get_height())
print('x ', ax1.patches[0].get_x())
print('y ', ax1.patches[0].get_y())
print()
print('width ', ax1.patches[1].get_width())
print('height ', ax1.patches[1].get_height())
print('x ', ax1.patches[1].get_x())
print('y ', ax1.patches[1].get_y())
print()
print('width ', ax1.patches[2].get_width())
print('height ', ax1.patches[2].get_height())
print('x ', ax1.patches[2].get_x())
print('y ', ax1.patches[2].get_y())

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
sns.barplot(ax=ax0, data=rcpo_df_norm, order=rcpo_prim_strict_df_norm.mean().sort_values(ascending=False).index.tolist())
ax0.tick_params(axis='x', labelrotation=90, labelsize=7.0)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
div_drawn_width_by=2
#-------------------------
n_patches_beg = len(ax0.patches)
sns.barplot(
    ax=ax0, 
    data=rcpo_df_norm, 
    order=rcpo_prim_strict_df_norm.mean().sort_values(ascending=False).index.tolist(), 
    color='red', 
    alpha=0.25, 
    label="Outages (All Xfmrs)"
)
n_patches_end = len(ax0.patches)
relative_position_idx=0
ax0 = Plot_Bar.adjust_bar_and_line_positions_and_widths(ax0, div_drawn_width_by, relative_position_idx, 
                                                        i_patch_beg=n_patches_beg, i_patch_end=n_patches_end)
#-------------------------
n_patches_beg = len(ax0.patches)
sns.barplot(
    ax=ax0, 
    data=rcpo_prim_strict_df_norm, 
    order=rcpo_prim_strict_df_norm.mean().sort_values(ascending=False).index.tolist(), 
    color='green', 
    alpha=0.25, 
    label="Outages (Xfmr tied to Outg. Location ID)"
)
n_patches_end = len(ax0.patches)
relative_position_idx=1
ax0 = Plot_Bar.adjust_bar_and_line_positions_and_widths(ax0, div_drawn_width_by, relative_position_idx, 
                                                        i_patch_beg=n_patches_beg, i_patch_end=n_patches_end)
#-------------------------
ax0.tick_params(axis='x', labelrotation=90, labelsize=7.0)
ax0.legend()

In [ ]:
common_barplot_kwargs = dict(alpha=0.25, fill=True, edgecolor='black', hatch='//')
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
reason_order = rcpo_prim_strict_df_norm.mean().sort_values(ascending=False).index.tolist()
n_reason_to_include=15
ax0 = Plot_Bar.plot_multiple_barplots(
    ax=ax0, 
    dfs_w_args=[
        (rcpo_df_norm,             {**dict(facecolor='red', hatch=None, label="Outages (All Xfmrs)"), **common_barplot_kwargs}), 
        (rcpo_prim_strict_df_norm, {**dict(facecolor='green', hatch=None, label="Outages (Xfmr tied to Outg. Location ID)"), **common_barplot_kwargs})
    ], 
    order=reason_order, 
    n_bars_to_include=n_reason_to_include, 
    keep_edges_opaque=True, 
    include_hatches=False, 
    draw_side_by_side=True, 
    draw_single_idx_full_width=None,     
    run_set_general_plotting_args=True, 
    orient='v',
    draw_legend=False, 
    legend_args=dict(fontsize=15), 
    title_args=dict(label=f'Title', fontsize=20), 
    ylabel_args = dict(ylabel=f'y-label', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
    xlabel_args = dict(xlabel='x-label', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
    tick_args=[dict(axis='x', labelrotation=90, labelsize=7), 
              dict(axis='y', labelsize=15)]
)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
reason_order = rcpo_prim_strict_df_norm.mean().sort_values(ascending=False).index.tolist()
n_reason_to_include=15
ax0 = Plot_Bar.plot_multiple_barplots(
    ax=ax0, 
    dfs_w_args=[rcpo_df_norm, rcpo_prim_strict_df_norm, rcpo_prim_strict_df_norm], 
    order=reason_order, 
    n_bars_to_include=n_reason_to_include, 
    keep_edges_opaque=True, 
    include_hatches=False, 
    draw_side_by_side=True, 
    draw_single_idx_full_width=2,     
    run_set_general_plotting_args=True, 
    draw_legend=False, 
    legend_args=dict(fontsize=15), 
    title_args=dict(label=f'Title', fontsize=20), 
    ylabel_args = dict(ylabel=f'y-label', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
    xlabel_args = dict(xlabel='x-label', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
    tick_args=[dict(axis='x', labelrotation=90, labelsize=7), 
              dict(axis='y', labelsize=15)]
)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
reason_order = rcpo_prim_strict_df_norm.mean().sort_values(ascending=False).index.tolist()
n_reason_to_include=15
ax0 = Plot_Bar.plot_multiple_barplots(
    ax=ax0, 
    dfs_w_args=[
        rcpo_df_norm, 
        rcpo_df_norm_by_outg_nSNs, 
        rcpo_prim_strict_df_norm, 
        rcpo_prim_strict_df_norm_by_outg_nSNs, 
        rcpo_no_outg_df_norm
    ], 
    order=reason_order, 
    n_bars_to_include=n_reason_to_include, 
    keep_edges_opaque=True, 
    include_hatches=False, 
    draw_side_by_side=True, 
    draw_single_idx_full_width=None,     
    run_set_general_plotting_args=True, 
    draw_legend=False, 
    legend_args=dict(fontsize=15), 
    title_args=dict(label=f'Title', fontsize=20), 
    ylabel_args = dict(ylabel=f'y-label', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
    xlabel_args = dict(xlabel='x-label', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
    tick_args=[dict(axis='x', labelrotation=90, labelsize=7), 
              dict(axis='y', labelsize=15)]
)

In [ ]:
common_barplot_kwargs = dict(alpha=0.25, fill=True, edgecolor='black')
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
reason_order = rcpo_prim_strict_df_norm.mean().sort_values(ascending=False).index.tolist()
n_reason_to_include=15
ax0 = Plot_Bar.plot_multiple_barplots(
    ax=ax0, 
    dfs_w_args=[
        (rcpo_df_norm,              {**dict(facecolor='red', hatch='//', label="Outages (All Xfmrs) - EndEvents nSNs Norm"), **common_barplot_kwargs}), 
        (rcpo_df_norm_by_outg_nSNs, {**dict(facecolor='red', hatch='-', label="Outages (All Xfmrs) - Outage nSNs Norm"), **common_barplot_kwargs}), 
        (rcpo_prim_strict_df_norm,              {**dict(facecolor='green', hatch='//', label="Outages (Xfmr tied to Outg. Location ID) - EndEvents nSNs Norm"), **common_barplot_kwargs}), 
        (rcpo_prim_strict_df_norm_by_outg_nSNs, {**dict(facecolor='green', hatch='-', label="Outages (Xfmr tied to Outg. Location ID) - Outage nSNs Norm"), **common_barplot_kwargs}), 
        (rcpo_no_outg_df_norm, {**dict(facecolor='orange', hatch=None, label="No Outages"), **common_barplot_kwargs})
    ], 
    order=reason_order, 
    n_bars_to_include=n_reason_to_include, 
    keep_edges_opaque=True, 
    include_hatches=False, 
    draw_side_by_side=True, 
    draw_single_idx_full_width=None,     
    run_set_general_plotting_args=True, 
    draw_legend=True, 
    legend_args=dict(fontsize=15), 
    title_args=dict(label=f'End Events', fontsize=20), 
    ylabel_args = dict(ylabel=f'Normalized Counts', fontsize=20, x=0.0, y=0.4, ha='left', va='bottom'), 
    xlabel_args = dict(xlabel='Reason', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
    tick_args=[dict(axis='x', labelrotation=90, labelsize=15), 
              dict(axis='y', labelsize=15)]
)

In [ ]:
common_barplot_kwargs = dict(alpha=0.25, fill=True, edgecolor='black')
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
reason_order = rcpo_prim_strict_df_norm.mean().sort_values(ascending=False).index.tolist()
n_reason_to_include=15
ax0 = Plot_Bar.plot_multiple_barplots(
    ax=ax0, 
    dfs_w_args=[
        (rcpo_df_norm,              {**dict(facecolor='red', hatch='//', label="Outages (All Xfmrs) - EndEvents nSNs Norm"), **common_barplot_kwargs}), 
        (rcpo_df_norm_by_outg_nSNs, {**dict(facecolor='red', hatch='-', label="Outages (All Xfmrs) - Outage nSNs Norm"), **common_barplot_kwargs}), 
        (rcpo_prim_strict_df_norm,              {**dict(facecolor='green', hatch='//', label="Outages (Xfmr tied to Outg. Location ID) - EndEvents nSNs Norm"), **common_barplot_kwargs}), 
        (rcpo_prim_strict_df_norm_by_outg_nSNs, {**dict(facecolor='green', hatch='-', label="Outages (Xfmr tied to Outg. Location ID) - Outage nSNs Norm"), **common_barplot_kwargs}), 
        (rcpo_no_outg_df_norm, {**dict(facecolor='orange', hatch=None, label="No Outages"), **common_barplot_kwargs})
    ], 
    order=reason_order, 
    n_bars_to_include=n_reason_to_include, 
    keep_edges_opaque=True, 
    include_hatches=False, 
    draw_side_by_side=True, 
    draw_single_idx_full_width=None,     
    run_set_general_plotting_args=True, 
    draw_legend=True, 
    legend_args=dict(fontsize=15), 
    title_args=dict(label=f'End Events', fontsize=20), 
    ylabel_args = dict(ylabel=f'Normalized Counts', fontsize=20, x=0.0, y=0.4, ha='left', va='bottom'), 
    xlabel_args = dict(xlabel='Reason', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
    tick_args=[dict(axis='x', labelrotation=90, labelsize=15), 
              dict(axis='y', labelsize=15)], 
    orient='h'
)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
ax0 = Plot_Bar.plot_barplot(
    ax=ax0, 
    df = rcpo_no_outg_df_norm, 
    order=rcpo_no_outg_df_norm.mean().sort_values(ascending=False).index.tolist(), 
    n_bars_to_include=10, 
    tick_args =dict(axis='x', labelrotation=90, labelsize=15.0)
)

In [ ]:
#barplot_kwargs = dict(alpha=0.5, fill=True, facecolor='red', edgecolor='black', hatch='//', palette='tab10')
barplot_kwargs = dict(alpha=0.5, fill=True, color='red', edgecolor='black', palette='tab10')

sns.set_theme(style="whitegrid")
tips = sns.load_dataset("tips")
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
n_patches_beg = len(ax0.patches)
sns.barplot(ax=ax0, x="day", y="total_bill", data=tips, **barplot_kwargs)

div_drawn_width_by=2
n_patches_end = len(ax0.patches)
relative_position_idx=1
ax0 = Plot_Bar.adjust_bar_and_line_positions_and_widths(ax0, div_drawn_width_by, relative_position_idx, 
                                                        i_patch_beg=n_patches_beg, i_patch_end=n_patches_end)
ax0.grid(True)

In [ ]:
#barplot_kwargs = dict(alpha=0.5, fill=True, facecolor='red', edgecolor='black', hatch='//', palette='tab10')
barplot_kwargs = dict(alpha=0.5, fill=True, color='red', edgecolor='black', palette='tab10')

sns.set_theme(style="whitegrid")
tips = sns.load_dataset("tips")
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
n_patches_beg = len(ax0.patches)
sns.barplot(ax=ax0, y="day", x="total_bill", data=tips, **barplot_kwargs, orient='h')

div_drawn_width_by=2
n_patches_end = len(ax0.patches)
relative_position_idx=1
ax0 = Plot_Bar.adjust_bar_and_line_positions_and_widths(ax0, div_drawn_width_by, relative_position_idx, 
                                                        i_patch_beg=n_patches_beg, i_patch_end=n_patches_end, orient='h')
ax0.grid(True)

In [ ]:
# assert(Utilities.is_object_one_of_types(grid_args, [bool, dict, list, tuple]))

In [ ]:
common_barplot_kwargs = dict(alpha=0.25, fill=True, edgecolor='black')
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
reason_order = rcpo_prim_strict_df_norm.mean().sort_values(ascending=False).index.tolist()
n_reason_to_include=15
ax0 = Plot_Bar.plot_multiple_barplots(
    ax=ax0, 
    dfs_w_args=[
        (rcpo_prim_strict_df_norm,              {**dict(facecolor='green', hatch='//', label="Outages (Xfmr tied to Outg. Location ID) - EndEvents nSNs Norm"), **common_barplot_kwargs}), 
        (rcpo_prim_strict_df_norm_by_outg_nSNs, {**dict(facecolor='green', hatch='-', label="Outages (Xfmr tied to Outg. Location ID) - Outage nSNs Norm"), **common_barplot_kwargs}), 
        (rcpo_prim_strict_df_norm_by_prim_nSNs, {**dict(facecolor='green', hatch='\\\\', label="Outages (Xfmr tied to Outg. Location ID) - Direct nSNs Norm"), **common_barplot_kwargs})
    ], 
    order=reason_order, 
    n_bars_to_include=n_reason_to_include, 
    keep_edges_opaque=True, 
    include_hatches=False, 
    draw_side_by_side=True, 
    draw_single_idx_full_width=None,     
    run_set_general_plotting_args=True, 
    draw_legend=True, 
    legend_args=dict(fontsize=15), 
    title_args=dict(label=f'End Events', fontsize=20), 
    ylabel_args = dict(ylabel=f'Normalized Counts', fontsize=20, x=0.0, y=0.4, ha='left', va='bottom'), 
    xlabel_args = dict(xlabel='Reason', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
    tick_args=[dict(axis='x', labelrotation=90, labelsize=15), 
              dict(axis='y', labelsize=15)]
)

In [ ]:
dfs_w_args = [rcpo_df_norm, rcpo_prim_strict_df_norm]
dfs_color_palette=None
include_hatches=False

In [ ]:
colors = Plot_General.get_standard_colors(len(dfs_w_args), palette=dfs_color_palette)
hatches = Plot_General.get_standard_hatches(len(dfs_w_args))
for i in range(len(dfs_w_args)):
    # First, normalize dfs_w_args[i] so it is a tuple whose first first element
    # is the pd.DataFrame and second is a dict containing the kwargs
    assert(Utilities.is_object_one_of_types(dfs_w_args[i], [pd.DataFrame, list, tuple]))
    if isinstance(dfs_w_args[i], pd.DataFrame):
        dfs_w_args[i] = (dfs_w_args[i], dict())
        
    # At this point, dfs_w_args[i] is definitely a list/tuple.  
    # Make sure length=2, first item is pd.DataFrame, and second is dict
    assert(len(dfs_w_args[i])==2 and 
           isinstance(dfs_w_args[i][0], pd.DataFrame) and 
           isinstance(dfs_w_args[i][1], dict))
    
    # Now, set all default values for plot_kwargs_i
    dfs_w_args[i][1]['hatch'] = dfs_w_args[i][1].get('hatch', hatches[i] if include_hatches else None)
    dfs_w_args[i][1]['color'] = dfs_w_args[i][1].get('color', colors[i])
    #dfs_w_args[i][1]['alpha'] = dfs_w_args[i][1].get('alpha', 1.0/len(dfs_w_args))
    dfs_w_args[i][1]['label'] = dfs_w_args[i][1].get('label', f'{i}')

In [ ]:
type(dfs_w_args[0][0])

In [ ]:
dfs_w_args[0][1]

In [ ]:
dfs_w_args[1][1]

# END: STANDARD PLOTTING STUFF

# BEGIN: DEVELOPMENTAL STUFF

In [ ]:
shared = [x for x in rcpo_prim_strict_df_OG.index if x in rcpo_df_OG.index]

In [ ]:
shared

In [ ]:
# rcpo_df_OG.loc[[shared[0]]]

In [ ]:
# rcpo_prim_strict_df_OG.loc[[shared[0]]]

In [ ]:
# def get_direct_SNs_in_outg()

In [ ]:
xfmr_equip_typ_nms_of_interest = ['TRANSFORMER, OH', 'TRANSFORMER, UG']
# df_outage_prim_strict = df_outage[(df_outage['LOCATION_ID']==df_outage['trsf_pole_nb']) & 
#                                   (df_outage['EQUIP_TYP_NM'].isin(xfmr_equip_typ_nms_of_interest))].copy()

In [ ]:
#FUCK

In [ ]:
# outg_rec_nb = [shared[0]]

# outg_rec_nb = rcpo_df_OG.index.tolist()
# dev_rcpo_df = rcpo_df_OG.copy()

outg_rec_nb = rcpo_prim_strict_df_OG.index.tolist()
dev_rcpo_df = rcpo_prim_strict_df_OG.copy()

dev_rcpo_df = MECPODf.project_level_0_columns_from_rcpo_wide(rcpo_df_wide=dev_rcpo_df, level_0_val='counts', droplevel=True)

# TODO!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Look into OFF_TM (in premise dimension of outages).
In most cases, this should be the same for all premises in an outage.
But what if it's not?

In [ ]:
start=time.time()
direct_SNs_in_outgs = DOVSOutages.build_direct_SNs_in_outgs_df(outg_rec_nbs=outg_rec_nb, use_exploded_method=True)
print(time.time()-start)

In [ ]:
direct_SNs_in_outgs

In [ ]:
# TODO!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# What are TRANSMISSION trsf_pole_nb?
direct_SNs_in_outgs[direct_SNs_in_outgs['trsf_pole_nb']=='TRANSMISSION']

In [ ]:
direct_SNs_in_outgs[direct_SNs_in_outgs['EQUIP_TYP_NM'].isin(xfmr_equip_typ_nms_of_interest)].equals(direct_SNs_in_outgs)

In [ ]:
outgs_w_prims = direct_SNs_in_outgs[direct_SNs_in_outgs['EQUIP_TYP_NM'].isin(xfmr_equip_typ_nms_of_interest)].index

In [ ]:
print(len(outgs_w_prims))
print(len(direct_SNs_in_outgs))

In [ ]:
direct_SNs_in_outgs

In [ ]:
dev_rcpo_df

In [ ]:
# dev_rcpo_df.loc['12048233']['_SNs']

In [ ]:
# direct_SNs_in_outgs.loc['12048233']['mfr_devc_ser_nbr'].tolist()

In [ ]:
# These are basically outages which do not have any "direct" serial numbers
missing_outg_rec_nbs = list(set(dev_rcpo_df.index).symmetric_difference(set(direct_SNs_in_outgs.index)))
print(len(missing_outg_rec_nbs))
missing_outg_rec_nbs

In [ ]:
dev_rcpo_df.loc[missing_outg_rec_nbs]

In [ ]:
dev_rcpo_df = MECPODf.add_outage_SNs_to_rcpo_df(dev_rcpo_df, include_outage_premise_nbs=False)

In [ ]:
dev_rcpo_df

In [ ]:
direct_SNs_in_outgs

In [ ]:
type(('a','b'))

In [ ]:
start = time.time()
dev_rcpo_df=MECPODf.add_prim_SNs_to_rcpo_df(
    rcpo_df=dev_rcpo_df, 
    direct_SNs_in_outgs_df=direct_SNs_in_outgs, 
    outg_rec_nb_col='index', 
    prim_SNs_col='direct_serial_numbers', 
    set_prim_nSNs=True
)
print(time.time()-start)

In [ ]:
dev_rcpo_df

In [ ]:
dev_rcpo_df_2a = dev_rcpo_df.drop(columns=['_prim_SNs', '_prim_nSNs']).copy()
#dev_rcpo_df_2a = dev_rcpo_df.drop(columns=[('counts', '_prim_SNs'), ('counts', '_prim_nSNs')]).copy()
start = time.time()
dev_rcpo_df_2a=MECPODf.add_prim_SNs_to_rcpo_df(
    rcpo_df=dev_rcpo_df_2a, 
    direct_SNs_in_outgs_df=None, 
    outg_rec_nb_col='index', 
    prim_SNs_col='direct_serial_numbers', 
    set_prim_nSNs=True
)
print(time.time()-start)
dev_rcpo_df_2a.equals(dev_rcpo_df)

In [ ]:
# dev_rcpo_df_2b = dev_rcpo_df.drop(columns=[('counts', '_prim_SNs'), ('counts', '_prim_nSNs')]).copy()
# dev_rcpo_df_2b[('counts', 'outg_rec_nb2')] = dev_rcpo_df_2b.index
dev_rcpo_df_2b = dev_rcpo_df.drop(columns=['_prim_SNs', '_prim_nSNs']).copy()
dev_rcpo_df_2b['outg_rec_nb2'] = dev_rcpo_df_2b.index
start = time.time()
dev_rcpo_df_2b=MECPODf.add_prim_SNs_to_rcpo_df(
    rcpo_df=dev_rcpo_df_2b, 
    direct_SNs_in_outgs_df=None, 
    outg_rec_nb_col='outg_rec_nb2', 
    prim_SNs_col='direct_serial_numbers', 
    set_prim_nSNs=True
)
print(time.time()-start)
dev_rcpo_df_2b[dev_rcpo_df.columns].equals(dev_rcpo_df)

In [ ]:
dev_rcpo_df

In [ ]:
dev_rcpo_df[dev_rcpo_df['_outg_nSNs']!=dev_rcpo_df['_prim_nSNs']]

In [ ]:
dev_rcpo_df[dev_rcpo_df['_nSNs']!=dev_rcpo_df['_prim_nSNs']]

In [ ]:
dev_rcpo_df

In [ ]:
dev_rcpo_df[dev_rcpo_df['_outg_nSNs']!=dev_rcpo_df['_prim_nSNs']]

In [ ]:
dev_rcpo_df[dev_rcpo_df['_nSNs']>dev_rcpo_df['_outg_nSNs']]

In [ ]:
rcpo_prim_strict_df_norm_by_outg_nSNs.head()

In [ ]:
rcpo_prim_strict_df_OG.head()

In [ ]:
dev_rcpo_df_raw_1 = rcpo_prim_strict_df_OG.copy()
dev_rcpo_df_raw_1 = MECPODf.project_level_0_columns_from_rcpo_wide(rcpo_df_wide=dev_rcpo_df_raw_1, level_0_val='counts', droplevel=True)

In [ ]:
dev_rcpo_df_raw_1[dev_rcpo_df_raw_1['Under Voltage (CA000400) cleared for meter']>0]

In [ ]:
start=time.time()
dev_rcpo_df_raw_1=MECPODf.build_rcpo_df_norm_by_outg_nSNs(dev_rcpo_df_raw_1)
print(time.time()-start)

In [ ]:
dev_rcpo_df_raw_1[dev_rcpo_df_raw_1['Under Voltage (CA000400) cleared for meter']>0]

In [ ]:
dev_rcpo_df_raw_2 = rcpo_prim_strict_df_OG.copy()
dev_rcpo_df_raw_2 = MECPODf.project_level_0_columns_from_rcpo_wide(rcpo_df_wide=dev_rcpo_df_raw_2, level_0_val='counts', droplevel=False)

In [ ]:
start=time.time()
dev_rcpo_df_raw_2=MECPODf.build_rcpo_df_norm_by_outg_nSNs(dev_rcpo_df_raw_2)
print(time.time()-start)

In [ ]:
dev_rcpo_df_raw_2.droplevel(0, axis=1).equals(dev_rcpo_df_raw_1)

In [ ]:
start=time.time()
dev_rcpo_df_raw_1=MECPODf.build_rcpo_df_norm_by_prim_nSNs(dev_rcpo_df_raw_1, direct_SNs_in_outgs_df=direct_SNs_in_outgs)
print(time.time()-start)

In [ ]:
start=time.time()
dev_rcpo_df_raw_2=MECPODf.build_rcpo_df_norm_by_prim_nSNs(dev_rcpo_df_raw_2, direct_SNs_in_outgs_df=direct_SNs_in_outgs)
print(time.time()-start)

In [ ]:
dev_rcpo_df_raw_2.droplevel(0, axis=1).equals(dev_rcpo_df_raw_1)

In [ ]:
dev_rcpo_df_raw_1

# END: DEVELOPMENTAL STUFF

In [ ]:
subset_dfs_dict = MECPODf.get_rcpo_df_subset_by_std_mjr_mnr_causes(rcpo_df_OG)
print(rcpo_df_OG.shape)
print(subset_dfs_dict['DL_OL'].shape)
print(subset_dfs_dict['DL_EQF'].shape)
print(subset_dfs_dict['DL_EQF_XFMR'].shape)

In [ ]:
common_barplot_kwargs = dict(alpha=0.25, fill=True, edgecolor='black')
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
reason_order = rcpo_prim_strict_df_norm.mean().sort_values(ascending=False).index.tolist()
n_reason_to_include=15
ax0 = Plot_Bar.plot_multiple_barplots(
    ax=ax0, 
    dfs_w_args=[
        (rcpo_df_norm_by_outg_nSNs, {**dict(facecolor='red', hatch='-', label="Outages (All Xfmrs) - Outage nSNs Norm"), **common_barplot_kwargs}), 
        (rcpo_prim_strict_df_norm_by_prim_nSNs, {**dict(facecolor='green', hatch='-', label="Outages (Xfmr tied to Outg. Location ID) - Direct Outage nSNs Norm"), **common_barplot_kwargs}), 
        (rcpo_no_outg_df_norm, {**dict(facecolor='orange', hatch=None, label="No Outages"), **common_barplot_kwargs})
    ], 
    order=reason_order, 
    n_bars_to_include=n_reason_to_include, 
    keep_edges_opaque=True, 
    include_hatches=False, 
    draw_side_by_side=True, 
    draw_single_idx_full_width=None,     
    run_set_general_plotting_args=True, 
    draw_legend=True, 
    legend_args=dict(fontsize=15), 
    title_args=dict(label=f'End Events', fontsize=20), 
    ylabel_args = dict(ylabel=f'Normalized Counts', fontsize=20, x=0.0, y=0.4, ha='left', va='bottom'), 
    xlabel_args = dict(xlabel='Reason', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
    tick_args=[dict(axis='x', labelrotation=90, labelsize=15), 
              dict(axis='y', labelsize=15)]
)

In [ ]:
subset_rcpo_dfs_dict = MECPODf.get_rcpo_df_subset_by_std_mjr_mnr_causes(rcpo_df_norm_by_outg_nSNs)
subset_rcpo_prim_strict_dfs_dict = MECPODf.get_rcpo_df_subset_by_std_mjr_mnr_causes(rcpo_prim_strict_df_norm_by_prim_nSNs)

In [ ]:
subset_rcpo_dfs_dict.keys()

In [ ]:
common_barplot_kwargs = dict(alpha=0.25, fill=True, edgecolor='black')
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
reason_order = rcpo_prim_strict_df_norm.mean().sort_values(ascending=False).index.tolist()
n_reason_to_include=10
ax0 = Plot_Bar.plot_multiple_barplots(
    ax=ax0, 
    dfs_w_args=[
        (rcpo_df_norm_by_outg_nSNs, {**dict(facecolor='red', hatch=None, label="Outages (All Xfmrs) - Outage nSNs Norm"), **common_barplot_kwargs}), 
        (subset_rcpo_dfs_dict['DL_OL'], {**dict(facecolor='red', hatch='//', label="DL_OL"), **common_barplot_kwargs}), 
        (subset_rcpo_dfs_dict['DL_EQF'], {**dict(facecolor='red', hatch='-', label="DL_EQF"), **common_barplot_kwargs}), 
        (subset_rcpo_dfs_dict['DL_EQF_XFMR'], {**dict(facecolor='red', hatch='\\\\', label="DL_EQF_XFMR"), **common_barplot_kwargs}), 
        (rcpo_prim_strict_df_norm_by_prim_nSNs, {**dict(facecolor='green', hatch=None, label="Outages (Xfmr tied to Outg. Location ID) - Direct Outage nSNs Norm"), **common_barplot_kwargs}),
        (subset_rcpo_prim_strict_dfs_dict['DL_OL'], {**dict(facecolor='green', hatch='//', label="DL_OL"), **common_barplot_kwargs}),
        (subset_rcpo_prim_strict_dfs_dict['DL_EQF'], {**dict(facecolor='green', hatch='-', label="DL_EQF"), **common_barplot_kwargs}),
        (subset_rcpo_prim_strict_dfs_dict['DL_EQF_XFMR'], {**dict(facecolor='green', hatch='\\\\', label="DL_EQF_XFMR"), **common_barplot_kwargs}),
        #(rcpo_no_outg_df_norm, {**dict(facecolor='orange', hatch=None, label="No Outages"), **common_barplot_kwargs})
    ], 
    order=reason_order, 
    n_bars_to_include=n_reason_to_include, 
    keep_edges_opaque=True, 
    include_hatches=False, 
    draw_side_by_side=True, 
    draw_single_idx_full_width=None,     
    run_set_general_plotting_args=True, 
    draw_legend=True, 
    legend_args=dict(fontsize=15), 
    title_args=dict(label=f'End Events', fontsize=20), 
    ylabel_args = dict(ylabel=f'Normalized Counts', fontsize=20, x=0.0, y=0.4, ha='left', va='bottom'), 
    xlabel_args = dict(xlabel='Reason', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
    tick_args=[dict(axis='x', labelrotation=90, labelsize=15), 
              dict(axis='y', labelsize=15)]
)

# DEVELOPMENT OF no outage normalization 

In [ ]:
def build_SNs_w_EDEs_df_from_local_files(
    date_0, 
    date_1, 
    opco='oh', 
    files_dir_base=r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\SNs_with_end_events', 
    allow_duplicate_dates=False, 
    drop_duplicate_dates=True
):
    r"""
    drop_duplicate_dates only has effect if allow_duplicate_dates is True, 
      in which case, any duplicate index entries will be dropped, with the first being kept
    Note: Cannot call drop_duplicates on DF because elements are lists
    
    Currently, FILES EXPECTED TO BE IN DIRECTORY os.path.join(files_dir_base, 'OPCO', opco)
    """
    #-------------------------
    files_dir = os.path.join(files_dir_base, 'OPCO', opco)
    assert(os.path.isdir(files_dir))
    #-------------------------
    year_0 = pd.to_datetime(date_0).year
    year_1 = pd.to_datetime(date_1).year
    years_needed = list(range(year_0,year_1+1))
    #-------------------------
    SNs_w_EDEs_dfs = []
    for year in years_needed:
        file_name = f'{year}.pkl'
        file_path = os.path.join(files_dir, file_name)
        assert(os.path.exists(file_path))
        #-----
        SNs_w_EDEs_df_i = pd.read_pickle(file_path)
        SNs_w_EDEs_dfs.append(SNs_w_EDEs_df_i)
    assert(len(SNs_w_EDEs_dfs)>0)
    if len(SNs_w_EDEs_dfs)==1:
        SNs_w_EDEs_df = SNs_w_EDEs_dfs[0]
    else:
        # Make sure all have same columns
        cols = SNs_w_EDEs_dfs[0].columns
        for df in SNs_w_EDEs_dfs:
            assert(df.columns==cols)
        #-----
        SNs_w_EDEs_df = pd.concat(SNs_w_EDEs_dfs)
        #-----
        if not allow_duplicate_dates:
            assert(SNs_w_EDEs_df.index.nunique()==SNs_w_EDEs_df.shape[0])
        else:
            if drop_duplicate_dates and SNs_w_EDEs_df.index.nunique()!=SNs_w_EDEs_df.shape[0]:
                SNs_w_EDEs_df = SNs_w_EDEs_df.groupby(SNs_w_EDEs_df.index).first()
                assert(SNs_w_EDEs_df.index.nunique()==SNs_w_EDEs_df.shape[0])
            else:
                # Not really necessary to check this...
                assert(SNs_w_EDEs_df.shape[0]==sum([x.shape[0] for x in SNs_w_EDEs_dfs]))
    #-------------------------
    # Make sure index is datetime with daily frequency
    SNs_w_EDEs_df.index = pd.to_datetime(SNs_w_EDEs_df.index)
    assert(pd.infer_freq(SNs_w_EDEs_df.index)=='D')
    SNs_w_EDEs_df.index.freq = 'D'

    # Make sure the index is sorted
    SNs_w_EDEs_df = SNs_w_EDEs_df.sort_index()
    #-------------------------
    return SNs_w_EDEs_df


def get_SNs_with_end_events_from_SNs_w_EDEs_df(
    SNs_w_EDEs_df, 
    date_0, 
    date_1, 
    SNs_col=None, 
    batch_size=None, 
    verbose=False
):
    r"""
    SNs_col:
      typically 'serialnumbers'
      If left as None, SNs_w_EDEs_df must have only a single column (which it typically does), and SNs_col
        is taken to be that column
    
    """
    #-------------------------
    if SNs_col is None:
        assert(SNs_w_EDEs_df.shape[1]==1)
        SNs_col = SNs_w_EDEs_df.columns[0]
    #-------------------------
    # Only date portion is considered here, since data taken daily
    # i.e., time is thrown away
    date_0 = pd.to_datetime(date_0).date()
    date_1 = pd.to_datetime(date_1).date()
    #-------------------------
    # Make sure index is datetime with daily frequency
    SNs_w_EDEs_df.index = pd.to_datetime(SNs_w_EDEs_df.index)
    assert(pd.infer_freq(SNs_w_EDEs_df.index)=='D')
    SNs_w_EDEs_df.index.freq = 'D'

    # Make sure the index is sorted
    SNs_w_EDEs_df = SNs_w_EDEs_df.sort_index()

    # Make sure date_0 and date_1 in bounds
    assert(pd.to_datetime(date_0)>=SNs_w_EDEs_df.index[0])
    assert(pd.to_datetime(date_1)<=SNs_w_EDEs_df.index[-1])

    sub_SNs_w_EDEs_df = SNs_w_EDEs_df[date_0:date_1]
    SNs = Utilities_df.consolidate_column_of_lists(
        df=sub_SNs_w_EDEs_df, 
        col=SNs_col, 
        sort=True, 
        include_None=False, 
        batch_size=batch_size, 
        verbose=verbose
    )
    #-------------------------
    return SNs

def get_SNs_with_end_events_from_local_files(
    date_0, 
    date_1, 
    opco='oh', 
    files_dir_base=r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\SNs_with_end_events', 
    batch_size=None, 
    verbose=False
):
    r"""
    """
    #-------------------------
    SNs_w_EDEs_df = build_SNs_w_EDEs_df_from_local_files(
        date_0=date_0, 
        date_1=date_1, 
        opco=opco, 
        files_dir_base=files_dir_base, 
        allow_duplicate_dates=False, 
        drop_duplicate_dates=True
    )
    #-------------------------
    SNs = get_SNs_with_end_events_from_SNs_w_EDEs_df(
        SNs_w_EDEs_df=SNs_w_EDEs_df, 
        date_0=date_0, 
        date_1=date_1,
        SNs_col=None, 
        batch_size=batch_size, 
        verbose=verbose
    )
    #-------------------------
    return SNs

In [ ]:
def build_mecpo_dict_from_pkls(
    pkls_base_dir, 
    days_min_max_outg_td_window, 
    normalize_by_time_interval=False
):
    r"""
    """
    #-------------------------
    mecpo = MECPOAn(
        days_min_max_outg_td_window=days_min_max_outg_td_window, 
        pkls_base_dir=pkls_base_dir, 
        is_no_outg=False, 
        normalize_by_time_interval=normalize_by_time_interval,
        naming_tag=None, 
        read_and_load_all_pickles=True
    )
    mecpo_prim_strict = MECPOAn(
        days_min_max_outg_td_window=days_min_max_outg_td_window, 
        pkls_base_dir=pkls_base_dir, 
        is_no_outg=False, 
        normalize_by_time_interval=normalize_by_time_interval,
        naming_tag='_prim_strict', 
        read_and_load_all_pickles=True
    )
    mecpo_no_outg = MECPOAn(
        days_min_max_outg_td_window=days_min_max_outg_td_window, 
        pkls_base_dir=pkls_base_dir, 
        is_no_outg=True, 
        normalize_by_time_interval=normalize_by_time_interval,
        naming_tag='_no_outg', 
        read_and_load_all_pickles=True
    )
    #-------------------------
    return {
        'full': mecpo, 
        'prim_strict': mecpo_prim_strict, 
        'no_outg': mecpo_no_outg
    }

In [ ]:
mecpo_dict_01_30_1 = build_mecpo_dict_from_pkls(
    r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\rcpo_dfs\NEW_w_prems', 
    days_min_max_outg_td_window=[1,30], normalize_by_time_interval=True
)

In [ ]:
# mecpo_dict_01_30_1 = build_mecpo_dict_from_pkls(
#     r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\rcpo_dfs', 
#     days_min_max_outg_td_window=[1,30], normalize_by_time_interval=normalize_by_time_interval
# )

# mecpo_dict_01_30_2 = build_mecpo_dict_from_pkls(
#     r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\rcpo_dfs\NEW_w_prems', 
#     days_min_max_outg_td_window=[1,30], normalize_by_time_interval=normalize_by_time_interval
# )

In [ ]:
files_dir_outg             = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\EndEvents'

is_no_outg = False
paths = Utilities.find_all_paths(base_dir=files_dir_outg, glob_pattern=file_path_glob)
# i_beg = 1000
# i_end = 2000
i_beg = 0
i_end = 1000
cols_and_types_to_convert_dict=None
to_numeric_errors='coerce'
assert_all_cols_equal=True
outg_rec_nb_col='outg_rec_nb'
addtnl_dropna_subset_cols=None

days_min_outg_td_window=1
days_max_outg_td_window=30

min_outg_td_window=datetime.timedelta(days=days_min_outg_td_window)
max_outg_td_window=datetime.timedelta(days=days_max_outg_td_window)

In [ ]:
dev_rcpo_df         = mecpo_dict_01_30_1['full'].cpo_dfs['rcpo_df_raw'].copy()
dev_rcpo_no_outg_df = mecpo_dict_01_30_1['no_outg'].cpo_dfs['rcpo_df_raw'].copy()

In [ ]:
run_infos_df = MECPOAn.get_bsln_time_interval_infos_df_from_summary_files(
    summary_paths=[AMIEndEvents.find_summary_file_from_csv(x) for x in paths], 
    output_prem_nbs_col='prem_nbs', 
    output_t_min_col='t_min', 
    output_t_max_col='DT_OFF_TS_FULL', 
    make_prem_nbs_idx=True, 
    include_summary_paths=True
)

In [ ]:
dev_rcpo_no_outg_df

In [ ]:
files_dir_no_outg             = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\EndEvents_NoOutg'
paths_no_outg = Utilities.find_all_paths(base_dir=files_dir_no_outg, glob_pattern=file_path_glob)

no_outg_time_infos_df = MECPOAn.get_bsln_time_interval_infos_df_from_summary_files(
    summary_paths=[AMIEndEvents.find_summary_file_from_csv(x) for x in paths_no_outg], 
    output_prem_nbs_col='prem_nbs', 
    output_t_min_col='t_min', 
    output_t_max_col='DT_OFF_TS_FULL', 
    make_prem_nbs_idx=True, 
    include_summary_paths=True
)

In [ ]:
no_outg_time_infos_df['t_min'].unique()

In [ ]:
dev_rcpo_df

In [ ]:
dev_rcpo_no_outg_df

In [ ]:
SNs_for_xfmrs_df = MeterPremise.get_SNs_for_xfmrs(trsf_pole_nbs=dev_rcpo_no_outg_df.index.tolist())

In [ ]:
SNs_for_xfmrs_df

In [ ]:
dev_rcpo_no_outg_df.head()

In [ ]:
is_datetime=False # Set to True if search should be datetime instead of date (i.e., if hours/minutes should be 
                  #   included or ignored in search)
prem_nbs_col_rcpo = '_prem_nbs'
prem_nbs_col_SNs_for_xfmrs = 'prem_nbs'
t_min_col = 't_min'
t_max_col = 'DT_OFF_TS_FULL'

In [ ]:
print(dev_rcpo_no_outg_df.shape[0])

In [ ]:
# start=time.time()
# prem_nbs_with_end_events_0 = get_SNs_with_end_events_from_local_files(
#     date_0='2017-01-01', 
#     date_1='2022-06-30', 
#     files_dir_base=r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\prem_nbs_with_end_events', 
# )
# print(time.time()-start)

In [ ]:
start=time.time()
prem_nbs_with_end_events = get_SNs_with_end_events_from_local_files(
    date_0='2017-01-01', 
    date_1='2022-06-30', 
    files_dir_base=r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\prem_nbs_with_end_events', 
    batch_size=100,
    verbose=True
)
print(time.time()-start)

In [ ]:
# prem_nbs_with_end_events_0==prem_nbs_with_end_events

In [ ]:
# rcpo_i = dev_rcpo_no_outg_df.iloc[4].copy() #pd.Series object
# prem_nbs_acquired_i = rcpo_i[prem_nbs_col_rcpo]
# #-----
# prem_nbs_in_xfmr_i = SNs_for_xfmrs_df.loc[rcpo_i.name][prem_nbs_col_SNs_for_xfmrs]
# #-----
# time_info_i = no_outg_time_infos_df.loc[prem_nbs_acquired_i]
# assert(time_info_i[t_min_col].nunique()==1)
# assert(time_info_i[t_max_col].nunique()==1)
# t_min_i = time_info_i.iloc[0][t_min_col]
# t_max_i = time_info_i.iloc[0][t_max_col]
# #-----
# # prem_nbs_with_end_events = get_SNs_with_end_events_from_local_files(
# #     date_0=t_min_i, 
# #     date_1=t_max_i, 
# #     files_dir_base=r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\prem_nbs_with_end_events'
# # )
# #-----
# prem_nbs_w_events_needed_i = [x for x in set(prem_nbs_in_xfmr_i).difference(set(prem_nbs_acquired_i)) if x in prem_nbs_with_end_events]

In [ ]:
prem_nbs_w_search_times = []
for idx in range(dev_rcpo_no_outg_df.shape[0]):
    #print(idx)
    if idx%50==0:
        print(idx)
    #TODO WHY AM I COPYING BELOW??! Don't think I need to
    rcpo_i = dev_rcpo_no_outg_df.iloc[idx].copy() #pd.Series object
    trsf_pole_nb_i = rcpo_i.name
    prem_nbs_acquired_i = rcpo_i[prem_nbs_col_rcpo]
    #-----
    if trsf_pole_nb_i not in SNs_for_xfmrs_df.index:
        print(f'trsf_pole_nb_i={trsf_pole_nb_i} not in SNs_for_xfmrs_df!!!!!')
        continue
    #-----
    prem_nbs_in_xfmr_i = SNs_for_xfmrs_df.loc[trsf_pole_nb_i][prem_nbs_col_SNs_for_xfmrs]
    #-----
    time_info_i = no_outg_time_infos_df.loc[prem_nbs_acquired_i]
    assert(time_info_i[t_min_col].nunique()==1)
    assert(time_info_i[t_max_col].nunique()==1)
    t_min_i = time_info_i.iloc[0][t_min_col]
    t_max_i = time_info_i.iloc[0][t_max_col]
    #-----
#     prem_nbs_with_end_events = get_SNs_with_end_events_from_local_files(
#         date_0=t_min_i, 
#         date_1=t_max_i, 
#         files_dir_base=r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\prem_nbs_with_end_events'
#     )
    #-----
    # Premise numbers needed are the difference between prem_nbs_in_xfmr_i and prem_nbs_acquired_i, but only if the premise numbers
    # have an end event
    # NOTE: Using set operations MUCH faster than list comprehension
    prem_nbs_w_events_needed_i = list((set(prem_nbs_in_xfmr_i).difference(set(prem_nbs_acquired_i)))
                                      .intersection(prem_nbs_with_end_events))
    #-----
    prem_nbs_w_search_times.append(dict(prem_nbs=prem_nbs_w_events_needed_i, t_min=t_min_i, t_max=t_max_i))

In [ ]:
# rcpo_i = dev_rcpo_no_outg_df.iloc[0].copy() #pd.Series object
# prem_nbs_acquired_i = rcpo_i[prem_nbs_col_rcpo]
# #-----
# prem_nbs_in_xfmr_i = SNs_for_xfmrs_df.loc[rcpo_i.name][prem_nbs_col_SNs_for_xfmrs]
# #-----
# time_info_i = no_outg_time_infos_df.loc[prem_nbs_acquired_i]
# assert(time_info_i[t_min_col].nunique()==1)
# assert(time_info_i[t_max_col].nunique()==1)
# t_min_i = time_info_i.iloc[0][t_min_col]
# t_max_i = time_info_i.iloc[0][t_max_col]
# #-----
# prem_nbs_with_end_events = get_SNs_with_end_events_from_local_files(
#     date_0=t_min_i, 
#     date_1=t_max_i, 
#     files_dir_base=r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\prem_nbs_with_end_events'
# )
# #-----
# prem_nbs_w_events_needed_i = [x for x in prem_nbs_in_xfmr_i if x in prem_nbs_with_end_events]
# #-----
# prem_nbs_w_search_times.append(dict(prem_nbs=prem_nbs_w_events_needed_i, t_min=t_min_i, t_max=t_max_i))

In [ ]:
prem_nbs_w_search_times

In [ ]:
pd.DataFrame(prem_nbs_w_search_times)['t_min']

In [ ]:
trsf_pole_nb_i

In [ ]:
len(prem_nbs_w_search_times)

In [ ]:
dev_rcpo_no_outg_df.index

In [ ]:
prem_nbs_w_search_times

In [ ]:
cols_of_interest_end_dev_event = TableInfos.AMIEndEvents_TI.std_columns_of_interest

cols_of_interest_met_prem = TableInfos.MeterPremise_TI.std_columns_of_interest
cols_of_interest_met_prem.append('curr_acct_cls_cd')

df_construct_type=DFConstructType.kRunSqlQuery
contstruct_df_args_end_events=None

build_sql_function = AMIEndEvents_SQL.build_sql_end_events

In [ ]:
# entry_i = prem_nbs_w_search_times[0]
# prem_nbs_i = entry_i['prem_nbs']
# t_min_i = entry_i['t_min']
# t_max_i = entry_i['t_max']

In [ ]:
print(len(prem_nbs_w_search_times))
for i in range(len(prem_nbs_w_search_times)):
    print(i)
    entry_i = prem_nbs_w_search_times[i]
    prem_nbs_i = entry_i['prem_nbs']
    t_min_i = entry_i['t_min']
    t_max_i = entry_i['t_max']
    #-------------------------
    build_sql_function_kwargs = dict(
        cols_of_interest=cols_of_interest_end_dev_event, 
        premise_nbs=prem_nbs_i, 
        serialnumber_col='serialnumber', 
        from_table_alias='un_rin', 
        schema_name='meter_events', 
        table_name='end_device_event', 
        datetime_col = 'valuesinterval', 
        datetime_pattern = r"([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*", 
        date_col     = 'aep_event_dt', 
        join_mp_args=dict(join_with_CTE=True, 
                          build_mp_kwargs=dict(cols_of_interest=cols_of_interest_met_prem), 
                          join_type='LEFT'), 
    #     field_to_split='df_mp_no_outg', 
    #     field_to_split_location_in_kwargs=['df_mp_no_outg'], 
    #     sort_coll_to_split=False,
    #     batch_size=10, verbose=True, n_update=1
    )

    #TODO!!!!!!!!!!!
    # NO OUTAGES use BOTH date and datetime.
    # This is WRONG, as it reduces the width of time by 1 day, since
    # datetime cuts off last day at 00:00:00, whereas date includes it
    # In any case, reproduce here so searching json files is consistent
    # with others.
    # ALSO, I think json wants t_min/max as string, not timestamp objects, hence
    # the conversion

    # if is_datetime:
    #     build_sql_function_kwargs['datetime_range'] = [t_min_i, t_max_i]
    # else:
    #     build_sql_function_kwargs['date_range'] = [t_min_i, t_max_i]

    build_sql_function_kwargs['datetime_range'] = [str(t_min_i), str(t_max_i)]
    build_sql_function_kwargs['date_range'] = [str(t_min_i), str(t_max_i)]

    #--------------------------------------------------
    save_args = dict(save_to_file=True, 
                     save_dir = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\EndEvents_NoOutg', 
                     save_name=r'end_events.csv', 
                     index=True)

    save_args['offset_int'] = GenAn.get_next_summary_file_tag_int(save_args)
    save_args = GenAn.prepare_save_args(save_args, make_save_dir_if_dne=False)
    save_args['save_name'] = Utilities.append_to_path(save_args['save_name'], appendix=f'_{save_args["offset_int"]}', 
                                                        ext_to_find=save_args['save_ext'], append_to_end_if_ext_no_found=True)
    save_args = GenAn.prepare_save_args(save_args, make_save_dir_if_dne=False)

    end_events = AMIEndEvents(
        df_construct_type=df_construct_type, 
        contstruct_df_args = contstruct_df_args_end_events, 
        build_sql_function=build_sql_function, 
        build_sql_function_kwargs=build_sql_function_kwargs, 
        init_df_in_constructor=True, 
        save_args=save_args
    )

In [ ]:
no_outg_time_infos_df = MECPOAn.get_bsln_time_interval_infos_df_from_summary_files(
    summary_paths=[AMIEndEvents.find_summary_file_from_csv(x) for x in paths[i_beg:i_end]], 
    output_prem_nbs_col='prem_nbs', 
    output_t_min_col='t_min', 
    output_t_max_col='DT_OFF_TS_FULL', 
    make_prem_nbs_idx=True, 
    include_summary_paths=True
)

In [ ]:
end_events_df_i = GenAn.read_df_from_csv_batch(
    paths=paths[i_beg:i_end], 
    cols_and_types_to_convert_dict=cols_and_types_to_convert_dict, 
    to_numeric_errors=to_numeric_errors, 
    make_all_columns_lowercase=True, 
    assert_all_cols_equal=assert_all_cols_equal
)
#-------------------------
# if end_events_df_i.shape[0]==0:
#     continue
#-------------------------
if (f'{outg_rec_nb_col}_gpd_for_sql' in end_events_df_i.columns and 
    outg_rec_nb_col not in end_events_df_i.columns):
    end_events_df_i = end_events_df_i.rename(columns={f'{outg_rec_nb_col}_gpd_for_sql':outg_rec_nb_col})
assert(outg_rec_nb_col in end_events_df_i.columns)
#-------------------------
dropna_subset_cols = [outg_rec_nb_col]
if addtnl_dropna_subset_cols is not None:
    dropna_subset_cols.extend(addtnl_dropna_subset_cols)
end_events_df_i = end_events_df_i.dropna(subset=dropna_subset_cols)

In [ ]:
no_outg_time_infos_df = MECPOAn.get_bsln_time_interval_infos_df_from_summary_files(
    summary_paths=[AMIEndEvents.find_summary_file_from_csv(x) for x in paths[i_beg:i_end]], 
    output_prem_nbs_col='prem_nbs', 
    output_t_min_col='t_min', 
    output_t_max_col='DT_OFF_TS_FULL', 
    make_prem_nbs_idx=True, 
    include_summary_paths=True
)

In [ ]:
no_outg_time_infos_df

In [ ]:
if not is_no_outg:
    # If min_outg_td_window or max_outg_td_window is not None, enforce time restrictions around outages
    if min_outg_td_window is not None or max_outg_td_window is not None:
        dovs_outgs = DOVSOutages(
            df_construct_type=DFConstructType.kRunSqlQuery, 
            contstruct_df_args=None, 
            init_df_in_constructor=True, 
            build_sql_function=DOVSOutages_SQL.build_sql_outage, 
            build_sql_function_kwargs=dict(
                outg_rec_nbs=end_events_df_i[outg_rec_nb_col].unique().tolist(), 
                from_table_alias='DOV', 
                datetime_col='DT_OFF_TS_FULL', 
                cols_of_interest=[
                    'OUTG_REC_NB', 
                    dict(field_desc=f"DOV.DT_ON_TS - DOV.STEP_DRTN_NB/(60*24)", 
                         alias='DT_OFF_TS_FULL', table_alias_prefix=None)
                ], 
                field_to_split='outg_rec_nbs'
            ),
        )
        outg_dt_off_df = dovs_outgs.df
        outg_dt_off_df = Utilities_df.convert_col_type(df=outg_dt_off_df, column='OUTG_REC_NB', to_type=str)
        outg_dt_off_df=outg_dt_off_df.set_index('OUTG_REC_NB')
        outg_dt_off_series = outg_dt_off_df['DT_OFF_TS_FULL']

        end_events_df_i=Utilities_dt.strip_tz_info_and_convert_to_dt(
            df=end_events_df_i, 
            time_col='valuesinterval', 
            placement_col='valuesinterval_local', 
            run_quick=True, 
            n_strip=6, 
            inplace=False
        )
        end_events_df_i = AMIEndEvents.enforce_end_events_within_interval_of_outage(
            end_events_df=end_events_df_i, 
            outg_times_series=outg_dt_off_series, 
            min_timedelta=min_outg_td_window, 
            max_timedelta=max_outg_td_window, 
            outg_rec_nb_col = outg_rec_nb_col, 
            datetime_col='valuesinterval_local'
        )

In [ ]:
SNs_in_outgs = DOVSOutages.get_serial_numbers_for_outages(
    outg_rec_nbs=outg_dt_off_df.index.unique().tolist(), 
    return_type=pd.Series, 
    col_type_outg_rec_nb=str, 
    col_type_premise_nb=None, 
    col_type_serial_nb=None, 
    return_premise_nbs_for_outages=True, 
)

In [ ]:
for outg_rec_nb in outg_dt_off_df.index.tolist():
    SNs_in_outg_i = SNs_in_outgs.loc[outg_rec_nb]['premise_nbs']

    # Actually searched 30 days before and after outage
    # (30 days before it began, and 30 days after it ended)
    # So, outg_time_i is 30 days after t_min
    search_time_window = datetime.timedelta(days=30)
    outg_time_i = no_outg_time_infos_df.loc[SNs_in_outg_i]['t_min']+search_time_window
    
    #NOPE, below not correct.  But good estimate if search_time_window unknown
#     outg_time_i = no_outg_time_infos_df.loc[SNs_in_outg_i]['t_min']+0.5*(
#         no_outg_time_infos_df.loc[SNs_in_outg_i]['DT_OFF_TS_FULL']-no_outg_time_infos_df.loc[SNs_in_outg_i]['t_min']
#     )


    # Possible more than one time, if premise suffered multiple outages
    outg_time_i=pd.to_datetime(outg_time_i.unique().tolist())

    # One, and only one, should be close to the outage time taken from outg_df_off_df
#     close_to_real_time = sum(abs(outg_time_i-outg_dt_off_df.loc[outg_rec_nb]['DT_OFF_TS_FULL'])<datetime.timedelta(minutes=10))
    close_to_real_time = sum(abs(outg_time_i-outg_dt_off_df.loc[outg_rec_nb]['DT_OFF_TS_FULL'])==datetime.timedelta(minutes=0))
    if close_to_real_time!=1:
        print(outg_rec_nb)
        print(f'close_to_real_time={close_to_real_time}')
        print()
    #assert(close_to_real_time==1)

In [ ]:
print(end_events_df_i['outg_rec_nb'].nunique())
outg_rec_nbs_w_meters_wo_events = []
for idx, df in end_events_df_i.groupby('outg_rec_nb'):
    SNs_in_outgs_i = SNs_in_outgs.loc[df['outg_rec_nb'].unique()]
    if len(SNs_in_outgs_i.iloc[0]['serial_numbers'])>df['serialnumber'].nunique():
        #print(idx)
        outg_rec_nbs_w_meters_wo_events.append(idx)

In [ ]:
len(outg_rec_nbs_w_meters_wo_events)

In [ ]:
SNs_in_outgs.loc[outg_rec_nbs_w_meters_wo_events[0]]

In [ ]:
end_events_df_i[end_events_df_i['outg_rec_nb']==outg_rec_nbs_w_meters_wo_events[0]]

In [ ]:
print(np.min(outg_dt_off_df['DT_OFF_TS_FULL']))
print(np.max(outg_dt_off_df['DT_OFF_TS_FULL']))

In [ ]:
# outg_rec_nb_i = outg_rec_nbs_w_meters_wo_events[100]
outg_rec_nb_i = '10413357'

In [ ]:
# NOT EXACTLY RIGHT!!!!!
t_max = outg_dt_off_df.loc[outg_rec_nb_i]['DT_OFF_TS_FULL']
t_min = t_max-datetime.timedelta(days=30)

sns_with_end_events = get_SNs_with_end_events_from_local_files(
    date_0=t_min, 
    date_1=t_max
)

# prem_nbs_with_end_events = get_SNs_with_end_events_from_local_files(
#     date_0=t_min, 
#     date_1=t_max, 
#     files_dir_base=r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\prem_nbs_with_end_events'
# )

prem_nbs_with_end_events = AMIEndEvents.get_end_event_distinct_fields(
    date_0=t_min, 
    date_1=t_max, 
    fields=['aep_premise_nb'], 
    are_datetime=True, 
    addtnl_build_sql_function_kwargs = dict(opco='oh')
)
prem_nbs_with_end_events=prem_nbs_with_end_events['aep_premise_nb'].tolist()

In [ ]:
prems_in_outg_i_from_DOVS = SNs_in_outgs.loc[outg_rec_nb_i]['premise_nbs']
prems_in_outg_i_from_end_events = end_events_df_i[end_events_df_i['outg_rec_nb']==outg_rec_nb_i]['aep_premise_nb'].unique().tolist()
expect_with_events = [x for x in prems_in_outg_i_from_DOVS
                      if x in prems_in_outg_i_from_end_events]

expect_no_events = [x for x in prems_in_outg_i_from_DOVS
                    if x not in prems_in_outg_i_from_end_events]  

print(len(set(prems_in_outg_i_from_end_events).difference(set(prems_in_outg_i_from_DOVS))))

In [ ]:
# Should be True
all_expected_found = all([x in prem_nbs_with_end_events for x in expect_with_events])

# Should be False
# REASON NOT FALSE: Likely because prem_nbs_with_end_events are only granularized to the date, not datetime
#                   Previously, when I was using AMIEndEvents.get_end_event_distinct_fields, this was due to the fact
#                     that are_datetime=False by default
#                   Currently, due to the fact that I am using get_SNs_with_end_events_from_local_files, which is daily
#                     by design.
any_unexpected_found = any([x in prem_nbs_with_end_events for x in expect_no_events])

In [ ]:
print(all_expected_found)
print(any_unexpected_found)

In [ ]:
expect_no_events

In [ ]:
[x for x in expect_no_events if x in prem_nbs_with_end_events]

In [ ]:
expect_with_events_2 = [x for x in SNs_in_outgs.loc[outg_rec_nb_i]['premise_nbs'] 
                        if x in mecpo_dict_01_30_1['full'].cpo_dfs['rcpo_df_OG'].loc[outg_rec_nb_i][('counts', '_prem_nbs')]]

expect_no_events_2 = [x for x in SNs_in_outgs.loc[outg_rec_nb_i]['premise_nbs'] 
                      if x not in mecpo_dict_01_30_1['full'].cpo_dfs['rcpo_df_OG'].loc[outg_rec_nb_i][('counts', '_prem_nbs')]]

In [ ]:
print(len(expect_with_events))
print(len(expect_with_events_2))

In [ ]:
print(len(expect_no_events))
print(len(expect_no_events_2))

In [ ]:
ME = AMIEndEvents(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    contstruct_df_args=None, 
    init_df_in_constructor=True,
    build_sql_function_kwargs = dict(
        cols_of_interest=TableInfos.AMIEndEvents_TI.std_columns_of_interest, 
        premise_nbs = expect_no_events, 
        datetime_range=[t_min, t_max], 
        #datetime_pattern = r"([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*"
    )
)

In [ ]:
[x for x in expect_no_events if x in prem_nbs_with_end_events]

In [ ]:
ME.df['aep_premise_nb'].nunique()

In [ ]:
print(f't_min = {t_min}')
print(f't_max = {t_max}')

In [ ]:
ME.df['valuesinterval'].min()

In [ ]:
ME.df['valuesinterval'].max()

In [ ]:
[x for x in expect_no_events if x in ME.df['aep_premise_nb'].tolist()]

In [ ]:
[x for x in expect_no_events if x in mecpo_dict_01_30_1['full'].cpo_dfs['rcpo_df_OG'].loc[outg_rec_nb_i][('counts', '_prem_nbs')]]

In [ ]:
ME.df[ME.df['aep_premise_nb'].isin(expect_no_events)]

In [ ]:
#FUCK
tmp_df = mecpo_dict_01_30_1['full'].cpo_dfs['rcpo_df_OG'].copy()

In [ ]:
tmp_lst = Utilities_df.consolidate_column_of_lists(df=tmp_df, col=('counts', '_prem_nbs'), sort=True, include_None=False)

In [ ]:
tmp_lst

In [ ]:
'076879016' in tmp_lst

In [ ]:
expect_no_events[0] in no_outg_time_infos_df.index

In [ ]:
print(f't_min = {t_min}')
print(f't_max = {t_max}')

In [ ]:
no_outg_time_infos_df.loc[expect_no_events]['summary_path'].unique()

In [ ]:
# print("WITH MP AS (\n\tSELECT\n\t\tmfr_devc_ser_nbr,\n\t\tlongitude,\n\t\tlatitude,\n\t\tstate_cd,\n\t\tprem_nb,\n\t\tsrvc_pole_nb,\n\t\ttrsf_pole_nb,\n\t\tlatitude_nb,\n\t\tlongitude_nb,\n\t\tcircuit_nb,\n\t\tcircuit_nm,\n\t\tstation_nb,\n\t\tstation_nm,\n\t\txfmr_nb,\n\t\tannual_kwh,\n\t\tannual_max_dmnd,\n\t\tmtr_stat_cd,\n\t\tmtr_stat_cd_desc,\n\t\tdevc_stat_cd,\n\t\tdevc_stat_cd_desc\n\tFROM default.meter_premise\n\tWHERE state_cd = 'OH'\n), \nUSG_gnrl AS (\n\tSELECT\n\t\tun_rin.issuertracking_id,\n\t\tun_rin.serialnumber,\n\t\tun_rin.enddeviceeventtypeid,\n\t\tun_rin.valuesinterval,\n\t\tun_rin.aep_premise_nb,\n\t\tun_rin.reason,\n\t\tun_rin.event_type,\n\t\tun_rin.aep_opco,\n\t\tun_rin.aep_event_dt,\n\t\tMP.*\n\tFROM meter_events.end_device_event un_rin\n\t\tINNER JOIN MP ON un_rin.serialnumber=MP.mfr_devc_ser_nbr\n\tWHERE un_rin.aep_opco = 'oh'\n\tAND   (\n\t\tun_rin.aep_event_dt BETWEEN '2018-03-24' AND '2018-05-23'\n\t)\n), \nUSG_0 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'10413189' AS OUTG_REC_NB_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2018-03-24' AND '2018-05-23'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '(\\\\d{4}-\\\\d{2}-\\\\d{2})T(\\\\d{2}:\\\\d{2}:\\\\d{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN '2018-03-24 11:20:00' AND '2018-05-23 15:46:00'\n\tAND   un_rin.aep_premise_nb IN ('071015994','071368586','071678586','072958586','073928586','074468586','074838586','074929665','075068586','075128586','075438586','070148586','070828586','073006763','073210519','073558586','075659294','075748586','076038586','076658586','076968586','077083059','077258586','077938586','078099338','078538586','078123652','078168586','078228586','078848586','079070098','079078586','079400346','079645451','079758586')\n), \nUSG_1 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'10413219' AS OUTG_REC_NB_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2018-03-24' AND '2018-05-23'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '(\\\\d{4}-\\\\d{2}-\\\\d{2})T(\\\\d{2}:\\\\d{2}:\\\\d{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN '2018-03-24 11:25:00' AND '2018-05-23 16:00:00'\n\tAND   un_rin.aep_premise_nb IN ('070831249','071655307','074755307','077855307')\n), \nUSG_2 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'10413222' AS OUTG_REC_NB_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2018-03-24' AND '2018-05-23'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '(\\\\d{4}-\\\\d{2}-\\\\d{2})T(\\\\d{2}:\\\\d{2}:\\\\d{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN '2018-03-24 14:00:00' AND '2018-05-23 15:55:00'\n\tAND   un_rin.aep_premise_nb IN ('078985144','079989129','070689129','078399129')\n), \nUSG_3 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'10413248' AS OUTG_REC_NB_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2018-03-24' AND '2018-05-23'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '(\\\\d{4}-\\\\d{2}-\\\\d{2})T(\\\\d{2}:\\\\d{2}:\\\\d{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN '2018-03-24 13:03:00' AND '2018-05-23 16:00:00'\n\tAND   un_rin.aep_premise_nb IN ('100157640','100373202','100108713','103257741','100465443','100527573','100679090','100951631','101065443','101127573','101171546','101539130','104227573','101969000','101991907','102118028','102569000','102717573','102833531','102849220','102879000','102965443','103627573','103668522','104079000','105669000','104304391','104912790','105565939','105817573','105861511','105877580','105949220','105979000','106140599','106417573','106461560','106579000','106665443','106746523','107017573','107153159','107179000','107239318','107265443','108018992','107327573','107811138','107992888','108106919','108175443','108263480','108369011','108473709','108512418','108769000','108792857','108837035','108917573','109217135','109369000','109679000','109929130','109765443','109827573','109837226')\n), \nUSG_4 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'10413295' AS OUTG_REC_NB_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2018-03-24' AND '2018-05-23'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '(\\\\d{4}-\\\\d{2}-\\\\d{2})T(\\\\d{2}:\\\\d{2}:\\\\d{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN '2018-03-24 12:54:00' AND '2018-05-23 16:16:00'\n\tAND   un_rin.aep_premise_nb IN ('102313930','102548237','103664597','105933146','108061371')\n), \nUSG_5 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'10413348' AS OUTG_REC_NB_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2018-03-24' AND '2018-05-23'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '(\\\\d{4}-\\\\d{2}-\\\\d{2})T(\\\\d{2}:\\\\d{2}:\\\\d{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN '2018-03-24 14:15:00' AND '2018-05-23 16:30:00'\n\tAND   un_rin.aep_premise_nb IN ('076081931','079320700')\n), \nUSG_6 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'10413354' AS OUTG_REC_NB_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2018-03-24' AND '2018-05-23'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '(\\\\d{4}-\\\\d{2}-\\\\d{2})T(\\\\d{2}:\\\\d{2}:\\\\d{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN '2018-03-24 15:33:00' AND '2018-05-23 16:55:00'\n\tAND   un_rin.aep_premise_nb IN ('076585374','078068327','073845230','074368327')\n), \nUSG_7 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'10413357' AS OUTG_REC_NB_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2018-03-24' AND '2018-05-23'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '(\\\\d{4}-\\\\d{2}-\\\\d{2})T(\\\\d{2}:\\\\d{2}:\\\\d{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN '2018-03-24 14:39:00' AND '2018-05-23 16:22:00'\n\tAND   un_rin.aep_premise_nb IN ('077851675','077898575','077909375','077911675','077958575','077975900','077989575','078029575','078050675','070009575','078110675','078139375','078141675','078188575','078201675','078279575','078298375','078339575','078360675','078391675','078409147','078420675','078449375','078451675','078498575','071870675','073471675','073500675','073529375','073531675','073578575','073622675','073669575','073688375','073690675','073729575','073750675','073781675','073810675','073839375','073841675','073888575','073901675','073948575','073998375','074019575','074029147','074040675','074069375','074071675','072509575','072519147','072530675','072559375','072561675','072619375','072621675','072668575','072685900','072699575','072712675','072759575','072778375','072780675','072819575','072840675','071909575','071930675','071959375','071961675','072001675','072032675','072079575','072098375','072139575','072160675','072191675','072249375','072251675','070569575','070588375','070590675','070629575','070650675','070681675','070710675','070739375','070788575','070801675','070848575','070898375','070939575','070960675','074902675','074949575','074968375','074970675','075008068','075010675','075039375','075041675','075088575','075101675','075136225','075179575','075198375','072298575','072309375','072311675','072358575','072375900','072389575','072402675','072449575','072470675','073469375','075239575','075291675','075309147','075320675','075349375','075351675','073002675','073049575','073070675','073074184','073109575','073119147','073130675','073159375','073161675','073219375','073221675','073268575','073285900','073299575','073312675','073359575','073378375','073380675','073419575','073440675','076678575','076722675','076769575','076788375','076790675','076829575','076850675','076879016','076881675','076892789','076910675','076939375','076941675','076988575','077012675','077059575','077078375','077119575','077129147','077140675','077169375','077171675','075880675','075919575','075940675','076058575','076149575','076170675','076209575','076219147','076230675','076259375','076261675','076319375','076321675','076368575','076385900','076399575','076412675','076459575','076478375','076480675','076519575','076540675','076561059','076571675','076600675','070991675','071000675','071029375','071031675','075398575','075409375','075411675','075458575','075475900','075489575','075502675','075549575','075570675','075574935','075609575','075619147','075630675','075659375','075661675','075719375','075721675','075768575','075785900','075812675','075859575','075878375','075971675','076009375','076011675','074598375','074639575','074660675','074691675','074709147','074720675','071341675','071388575','071401675','071479575','071498375','071539575','071560675','071591675','071609147','071620675','071649375','071651675','071698575','071709375','079559575','079580675','079619575','079640675','079671675','079700675','079729375','079731675','079778575','079822675','079847764','072209147','074100675','074129375','074131675','074178575','074195900','074222675','074259259','074269575','074288375','074290675','074329575','074350675','074381675','074439375','074441675','074488575','074501675','074548575','074579575','072871675','072900675','072929375','074749375','074751675','074809375','074811675','074858575','074875900','074889575','079512675','079516407','079523013','071169575','071188375','071190675','071229575','071250675','071281675','071310675','071339375','079869575','079888375','079890675','079929575','079950675','079981675','071711675','071758575','071771462','071775900','071789575','071802675','071849575','070119375','070121675','070185900','070199575','070212675','070259575','070278375','070280675','070319575','070340675','070371675','070400675','070429375','070431675','071078575','071095900','071122675','079109375','079111675','079158575','079189575','079202675','079238697','079249575','079270675','079309575','079319147','079330675','079359375','079361675','079419375','079421675','079468575','079485900','079499575','078509375','078511675','078558575','078575900','078589575','078602675','078649575','070019147','070030675','070059375','070061675','072940036','072978575','070445659','070478575','070495900','070522675','078670675','078709575','078719147','078730675','078759375','078761675','078819375','078821675','078868575','078885900','078899575','078912675','078959575','078978375','078980675','079011469','079020675','079049375','079051675','079098575','079100530','077739575','077760675','077791675','077809147','077820675','077849375','077200675','077229375','077231675','077278575','077295900','077299488','077322675','077369575','077388375','077390675','077429575','077450675','077481675','077510675','077539375','077541675','077588575','077601675','077648575','077685702','077698375','076629375','076631675')\n), \nUSG_8 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'10413371' AS OUTG_REC_NB_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2018-03-24' AND '2018-05-23'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '(\\\\d{4}-\\\\d{2}-\\\\d{2})T(\\\\d{2}:\\\\d{2}:\\\\d{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN '2018-03-24 14:00:00' AND '2018-05-23 16:10:00'\n\tAND   un_rin.aep_premise_nb = '070478794'\n), \nUSG_9 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'10413393' AS OUTG_REC_NB_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2018-03-24' AND '2018-05-23'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '(\\\\d{4}-\\\\d{2}-\\\\d{2})T(\\\\d{2}:\\\\d{2}:\\\\d{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN '2018-03-24 08:45:00' AND '2018-05-23 11:00:00'\n\tAND   un_rin.aep_premise_nb = '102474441'\n)\n\nSELECT * FROM USG_0\nUNION\nSELECT * FROM USG_1\nUNION\nSELECT * FROM USG_2\nUNION\nSELECT * FROM USG_3\nUNION\nSELECT * FROM USG_4\nUNION\nSELECT * FROM USG_5\nUNION\nSELECT * FROM USG_6\nUNION\nSELECT * FROM USG_7\nUNION\nSELECT * FROM USG_8\nUNION\nSELECT * FROM USG_9",)

In [ ]:
tmp_sql = """
WITH MP AS (
	SELECT
		mfr_devc_ser_nbr,
		longitude,
		latitude,
		state_cd,
		prem_nb,
		srvc_pole_nb,
		trsf_pole_nb,
		latitude_nb,
		longitude_nb,
		circuit_nb,
		circuit_nm,
		station_nb,
		station_nm,
		xfmr_nb,
		annual_kwh,
		annual_max_dmnd,
		mtr_stat_cd,
		mtr_stat_cd_desc,
		devc_stat_cd,
		devc_stat_cd_desc
	FROM default.meter_premise
	WHERE state_cd = 'OH'
), 
USG_gnrl AS (
	SELECT
		un_rin.issuertracking_id,
		un_rin.serialnumber,
		un_rin.enddeviceeventtypeid,
		un_rin.valuesinterval,
		un_rin.aep_premise_nb,
		un_rin.reason,
		un_rin.event_type,
		un_rin.aep_opco,
		un_rin.aep_event_dt,
		MP.*
	FROM meter_events.end_device_event un_rin
		LEFT JOIN MP ON un_rin.serialnumber=MP.mfr_devc_ser_nbr
	WHERE un_rin.aep_opco = 'oh'
	AND   (
		un_rin.aep_event_dt BETWEEN '2018-03-24' AND '2018-05-23'
	)
), 
USG_0 AS (
	SELECT
		un_rin.*,
		'10413189' AS OUTG_REC_NB_GPD_FOR_SQL
	FROM USG_gnrl un_rin
	WHERE un_rin.aep_event_dt BETWEEN '2018-03-24' AND '2018-05-23'
	AND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2018-03-24 11:20:00' AND TIMESTAMP '2018-05-23 15:46:00'
	AND   un_rin.aep_premise_nb IN ('071015994','071368586','071678586','072958586','073928586','074468586','074838586','074929665','075068586','075128586','075438586','070148586','070828586','073006763','073210519','073558586','075659294','075748586','076038586','076658586','076968586','077083059','077258586','077938586','078099338','078538586','078123652','078168586','078228586','078848586','079070098','079078586','079400346','079645451','079758586')
), 
USG_1 AS (
	SELECT
		un_rin.*,
		'10413219' AS OUTG_REC_NB_GPD_FOR_SQL
	FROM USG_gnrl un_rin
	WHERE un_rin.aep_event_dt BETWEEN '2018-03-24' AND '2018-05-23'
	AND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2018-03-24 11:25:00' AND TIMESTAMP '2018-05-23 16:00:00'
	AND   un_rin.aep_premise_nb IN ('070831249','071655307','074755307','077855307')
), 
USG_2 AS (
	SELECT
		un_rin.*,
		'10413222' AS OUTG_REC_NB_GPD_FOR_SQL
	FROM USG_gnrl un_rin
	WHERE un_rin.aep_event_dt BETWEEN '2018-03-24' AND '2018-05-23'
	AND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2018-03-24 14:00:00' AND TIMESTAMP '2018-05-23 15:55:00'
	AND   un_rin.aep_premise_nb IN ('078985144','079989129','070689129','078399129')
), 
USG_3 AS (
	SELECT
		un_rin.*,
		'10413248' AS OUTG_REC_NB_GPD_FOR_SQL
	FROM USG_gnrl un_rin
	WHERE un_rin.aep_event_dt BETWEEN '2018-03-24' AND '2018-05-23'
	AND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2018-03-24 13:03:00' AND TIMESTAMP '2018-05-23 16:00:00'
	AND   un_rin.aep_premise_nb IN ('100157640','100373202','100108713','103257741','100465443','100527573','100679090','100951631','101065443','101127573','101171546','101539130','104227573','101969000','101991907','102118028','102569000','102717573','102833531','102849220','102879000','102965443','103627573','103668522','104079000','105669000','104304391','104912790','105565939','105817573','105861511','105877580','105949220','105979000','106140599','106417573','106461560','106579000','106665443','106746523','107017573','107153159','107179000','107239318','107265443','108018992','107327573','107811138','107992888','108106919','108175443','108263480','108369011','108473709','108512418','108769000','108792857','108837035','108917573','109217135','109369000','109679000','109929130','109765443','109827573','109837226')
), 
USG_4 AS (
	SELECT
		un_rin.*,
		'10413295' AS OUTG_REC_NB_GPD_FOR_SQL
	FROM USG_gnrl un_rin
	WHERE un_rin.aep_event_dt BETWEEN '2018-03-24' AND '2018-05-23'
	AND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2018-03-24 12:54:00' AND TIMESTAMP '2018-05-23 16:16:00'
	AND   un_rin.aep_premise_nb IN ('102313930','102548237','103664597','105933146','108061371')
), 
USG_5 AS (
	SELECT
		un_rin.*,
		'10413348' AS OUTG_REC_NB_GPD_FOR_SQL
	FROM USG_gnrl un_rin
	WHERE un_rin.aep_event_dt BETWEEN '2018-03-24' AND '2018-05-23'
	AND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2018-03-24 14:15:00' AND TIMESTAMP '2018-05-23 16:30:00'
	AND   un_rin.aep_premise_nb IN ('076081931','079320700')
), 
USG_6 AS (
	SELECT
		un_rin.*,
		'10413354' AS OUTG_REC_NB_GPD_FOR_SQL
	FROM USG_gnrl un_rin
	WHERE un_rin.aep_event_dt BETWEEN '2018-03-24' AND '2018-05-23'
	AND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2018-03-24 15:33:00' AND TIMESTAMP '2018-05-23 16:55:00'
	AND   un_rin.aep_premise_nb IN ('076585374','078068327','073845230','074368327')
), 
USG_7 AS (
	SELECT
		un_rin.*,
		'10413357' AS OUTG_REC_NB_GPD_FOR_SQL
	FROM USG_gnrl un_rin
	WHERE un_rin.aep_event_dt BETWEEN '2018-03-24' AND '2018-05-23'
	AND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2018-03-24 14:39:00' AND TIMESTAMP '2018-05-23 16:22:00'
	AND   un_rin.aep_premise_nb IN ('077851675','077898575','077909375','077911675','077958575','077975900','077989575','078029575','078050675','070009575','078110675','078139375','078141675','078188575','078201675','078279575','078298375','078339575','078360675','078391675','078409147','078420675','078449375','078451675','078498575','071870675','073471675','073500675','073529375','073531675','073578575','073622675','073669575','073688375','073690675','073729575','073750675','073781675','073810675','073839375','073841675','073888575','073901675','073948575','073998375','074019575','074029147','074040675','074069375','074071675','072509575','072519147','072530675','072559375','072561675','072619375','072621675','072668575','072685900','072699575','072712675','072759575','072778375','072780675','072819575','072840675','071909575','071930675','071959375','071961675','072001675','072032675','072079575','072098375','072139575','072160675','072191675','072249375','072251675','070569575','070588375','070590675','070629575','070650675','070681675','070710675','070739375','070788575','070801675','070848575','070898375','070939575','070960675','074902675','074949575','074968375','074970675','075008068','075010675','075039375','075041675','075088575','075101675','075136225','075179575','075198375','072298575','072309375','072311675','072358575','072375900','072389575','072402675','072449575','072470675','073469375','075239575','075291675','075309147','075320675','075349375','075351675','073002675','073049575','073070675','073074184','073109575','073119147','073130675','073159375','073161675','073219375','073221675','073268575','073285900','073299575','073312675','073359575','073378375','073380675','073419575','073440675','076678575','076722675','076769575','076788375','076790675','076829575','076850675','076879016','076881675','076892789','076910675','076939375','076941675','076988575','077012675','077059575','077078375','077119575','077129147','077140675','077169375','077171675','075880675','075919575','075940675','076058575','076149575','076170675','076209575','076219147','076230675','076259375','076261675','076319375','076321675','076368575','076385900','076399575','076412675','076459575','076478375','076480675','076519575','076540675','076561059','076571675','076600675','070991675','071000675','071029375','071031675','075398575','075409375','075411675','075458575','075475900','075489575','075502675','075549575','075570675','075574935','075609575','075619147','075630675','075659375','075661675','075719375','075721675','075768575','075785900','075812675','075859575','075878375','075971675','076009375','076011675','074598375','074639575','074660675','074691675','074709147','074720675','071341675','071388575','071401675','071479575','071498375','071539575','071560675','071591675','071609147','071620675','071649375','071651675','071698575','071709375','079559575','079580675','079619575','079640675','079671675','079700675','079729375','079731675','079778575','079822675','079847764','072209147','074100675','074129375','074131675','074178575','074195900','074222675','074259259','074269575','074288375','074290675','074329575','074350675','074381675','074439375','074441675','074488575','074501675','074548575','074579575','072871675','072900675','072929375','074749375','074751675','074809375','074811675','074858575','074875900','074889575','079512675','079516407','079523013','071169575','071188375','071190675','071229575','071250675','071281675','071310675','071339375','079869575','079888375','079890675','079929575','079950675','079981675','071711675','071758575','071771462','071775900','071789575','071802675','071849575','070119375','070121675','070185900','070199575','070212675','070259575','070278375','070280675','070319575','070340675','070371675','070400675','070429375','070431675','071078575','071095900','071122675','079109375','079111675','079158575','079189575','079202675','079238697','079249575','079270675','079309575','079319147','079330675','079359375','079361675','079419375','079421675','079468575','079485900','079499575','078509375','078511675','078558575','078575900','078589575','078602675','078649575','070019147','070030675','070059375','070061675','072940036','072978575','070445659','070478575','070495900','070522675','078670675','078709575','078719147','078730675','078759375','078761675','078819375','078821675','078868575','078885900','078899575','078912675','078959575','078978375','078980675','079011469','079020675','079049375','079051675','079098575','079100530','077739575','077760675','077791675','077809147','077820675','077849375','077200675','077229375','077231675','077278575','077295900','077299488','077322675','077369575','077388375','077390675','077429575','077450675','077481675','077510675','077539375','077541675','077588575','077601675','077648575','077685702','077698375','076629375','076631675')
), 
USG_8 AS (
	SELECT
		un_rin.*,
		'10413371' AS OUTG_REC_NB_GPD_FOR_SQL
	FROM USG_gnrl un_rin
	WHERE un_rin.aep_event_dt BETWEEN '2018-03-24' AND '2018-05-23'
	AND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2018-03-24 14:00:00' AND TIMESTAMP '2018-05-23 16:10:00'
	AND   un_rin.aep_premise_nb = '070478794'
), 
USG_9 AS (
	SELECT
		un_rin.*,
		'10413393' AS OUTG_REC_NB_GPD_FOR_SQL
	FROM USG_gnrl un_rin
	WHERE un_rin.aep_event_dt BETWEEN '2018-03-24' AND '2018-05-23'
	AND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2018-03-24 08:45:00' AND TIMESTAMP '2018-05-23 11:00:00'
	AND   un_rin.aep_premise_nb = '102474441'
)

SELECT * FROM USG_0
UNION
SELECT * FROM USG_1
UNION
SELECT * FROM USG_2
UNION
SELECT * FROM USG_3
UNION
SELECT * FROM USG_4
UNION
SELECT * FROM USG_5
UNION
SELECT * FROM USG_6
UNION
SELECT * FROM USG_7
UNION
SELECT * FROM USG_8
UNION
SELECT * FROM USG_9
"""

In [ ]:
end_events_df_i.columns

In [ ]:
tmp_df = pd.read_sql(tmp_sql, Utilities.get_athena_prod_aws_connection())

In [ ]:
tmp_df2 = pd.read_csv(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\EndEvents\end_events_1007.csv', dtype=str)

In [ ]:
print(tmp_df.shape)
print(tmp_df2.shape)

In [ ]:
tmp_df[tmp_df['mfr_devc_ser_nbr']=='676517633']

In [ ]:
tmp_df[tmp_df['serialnumber']=='876683636']

In [ ]:
tmp_df2[tmp_df2['serialnumber']=='876683636']

In [ ]:
len(set(tmp_df['mfr_devc_ser_nbr'].unique().tolist()).difference(set(tmp_df2['mfr_devc_ser_nbr'].unique().tolist())))

In [ ]:
tmp_df['mfr_devc_ser_nbr'].unique().tolist()[0]

In [ ]:
tmp_df2['mfr_devc_ser_nbr'].unique().tolist()[0]

In [ ]:
diff_SNs = set(tmp_df['mfr_devc_ser_nbr'].unique()).difference(set(tmp_df2['mfr_devc_ser_nbr'].unique()))

In [ ]:
tmp_df[tmp_df['mfr_devc_ser_nbr'].isin(diff_SNs)]

In [ ]:
tmp_df[tmp_df['serialnumber'].isin(set(tmp_df['serialnumber'].unique()).difference(set(tmp_df2['serialnumber'].unique())))][['serialnumber', 'mfr_devc_ser_nbr']]

In [ ]:
ME.get_sql().print()

In [ ]:
ME = AMIEndEvents(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    contstruct_df_args=None, 
    init_df_in_constructor=True,
    build_sql_function_kwargs = dict(
        cols_of_interest=TableInfos.AMIEndEvents_TI.std_columns_of_interest, 
        premise_nbs = expect_no_events, 
        date_range=[t_min, t_max+datetime.timedelta(days=30)], 
        datetime_range=[t_min, t_max+datetime.timedelta(days=30)], 
        #datetime_pattern = r"([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*"
    )
)

In [ ]:
[x for x in expect_no_events if x in ME.df['aep_premise_nb'].tolist()]

In [ ]:
[x for x in expect_no_events if x in tmp_df['aep_premise_nb'].tolist()]

In [ ]:
[x for x in expect_no_events if x in tmp_df2['aep_premise_nb'].tolist()]

In [ ]:
prem_nbs_with_end_events_2 = AMIEndEvents.get_end_event_distinct_fields(
    date_0=t_min, 
    date_1=t_max, 
    fields=['aep_premise_nb'], 
    are_datetime=True, 
    addtnl_build_sql_function_kwargs = dict(opco='oh')
)

In [ ]:
prem_nbs_with_end_events_2

In [ ]:
[x in prem_nbs_with_end_events_2['aep_premise_nb'].tolist() for x in expect_no_events]

In [ ]:
[x in prem_nbs_with_end_events_2['aep_premise_nb'].tolist() for x in expect_with_events]

In [ ]:
t_min

In [ ]:
t_max

In [ ]:
ME_2 = AMIEndEvents(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    contstruct_df_args=None, 
    init_df_in_constructor=True,
    build_sql_function_kwargs = dict(
        cols_of_interest=TableInfos.AMIEndEvents_TI.std_columns_of_interest, 
        #premise_nbs = expect_no_events, 
        datetime_range=[t_min, t_max], 
        #datetime_pattern = r"([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*"
    )
)

In [ ]:
ME_2.df[ME_2.df['aep_premise_nb'].isin(expect_no_events)]['valuesinterval'].min()

In [ ]:
ME_2.df[ME_2.df['aep_premise_nb'].isin(expect_no_events)]['valuesinterval'].max()

In [ ]:
ME_2.get_sql().print()

In [ ]:
assert(all_expected_found)
assert(not any_unexpected_found)

In [ ]:
#TODO WHY IS THIS NOT EMPTY?!
# BECAUSE I USE PREMISE NUMBER NOT SERIAL NUMBER?!
# YES! BECAUSE apparently meter_events has some serial numbers that meter_premise does not!!!!!
set(SNs_in_end_events_i).difference(set(SNs_in_outg_rec_nb_i))

In [ ]:
# # SIMPLIFIED!!!!!!!!!!!!!!!!!
# n_skipped=0
# print(len(outg_rec_nbs_w_meters_wo_events))
# for i,outg_rec_nb_i in enumerate(outg_rec_nbs_w_meters_wo_events):
# # for i,outg_rec_nb_i in enumerate(['10290769']):
#     if i%100==0:
#         print(i)
#     # NOT EXACTLY RIGHT!!!!!
#     t_max = outg_dt_off_df.loc[outg_rec_nb_i]['DT_OFF_TS_FULL']
#     t_min = t_max-datetime.timedelta(days=30)
    
#     if t_max.year>2022 or t_min.year<2018:
#         n_skipped += 1
#         continue
        
#     sns_with_end_events = get_SNs_with_end_events_from_local_files(
#         date_0=t_min, 
#         date_1=t_max
#     )
    
#     SNs_in_end_events_i = end_events_df_i[end_events_df_i['outg_rec_nb']==outg_rec_nb_i]['serialnumber'].unique().tolist()
#     if len(set(SNs_in_end_events_i).difference(set(sns_with_end_events)))>0:
#         print(outg_rec_nb_i)
        
# print(f'n_skipped = {n_skipped}')

In [ ]:
# # SIMPLIFIED!!!!!!!!!!!!!!!!!
# n_skipped=0
# print(len(outg_rec_nbs_w_meters_wo_events))
# for i,outg_rec_nb_i in enumerate(outg_rec_nbs_w_meters_wo_events):
# # for i,outg_rec_nb_i in enumerate(['10290769']):
#     if i%100==0:
#         print(i)
#     # NOT EXACTLY RIGHT!!!!!
#     t_max = outg_dt_off_df.loc[outg_rec_nb_i]['DT_OFF_TS_FULL']
#     t_min = t_max-datetime.timedelta(days=30)
    
#     if t_max.year>2022 or t_min.year<2018:
#         n_skipped += 1
#         continue
        
#     prem_nbs_with_end_events = get_SNs_with_end_events_from_local_files(
#         date_0=t_min, 
#         date_1=t_max, 
#         files_dir_base=r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\prem_nbs_with_end_events'
#     )
    
#     SNs_in_end_events_i = end_events_df_i[end_events_df_i['outg_rec_nb']==outg_rec_nb_i]['aep_premise_nb'].unique().tolist()
#     if len(set(SNs_in_end_events_i).difference(set(prem_nbs_with_end_events)))>0:
#         print(outg_rec_nb_i)
        
# print(f'n_skipped = {n_skipped}')

In [ ]:
len(sns_with_end_events)

In [ ]:
len(SNs_in_end_events_i)

In [ ]:
n_skipped=0
outg_rec_nbs_w_missing_SNs = []
outg_rec_nbs_wo_all_found = []
outg_rec_nbs_w_unexpected = []
print(len(outg_rec_nbs_w_meters_wo_events))
# for i,outg_rec_nb_i in enumerate(outg_rec_nbs_w_meters_wo_events):
for i,outg_rec_nb_i in enumerate(['10283904']):
    if i%100==0:
        print(i)
    # NOT EXACTLY RIGHT!!!!!
    t_max = outg_dt_off_df.loc[outg_rec_nb_i]['DT_OFF_TS_FULL']
    t_min = t_max-datetime.timedelta(days=30)
    
    if t_max.year>2022 or t_min.year<2018:
        n_skipped += 1
        continue

    sns_with_end_events = AMIEndEvents.get_end_event_distinct_fields(
        date_0=t_min, 
        date_1=t_max, 
        fields=['serialnumber'], 
        addtnl_build_sql_function_kwargs = dict(opco='oh')
    )


    # For each serial number in the outage (from SNs_in_outgs.loc[outg_rec_nb_i]['serial_numbers'])
    #   if serial number contained in end_events_df_i[end_events_df_i['outg_rec_nb']==outg_rec_nb_i], it is expected to have an event
    #     as determined now by sns_with_end_events
    #   Otherwise, not expected to have an event in sns_with_end_events
    SNs_in_outg_rec_nb_i = SNs_in_outgs.loc[outg_rec_nb_i]['serial_numbers']
    SNs_in_end_events_i = end_events_df_i[end_events_df_i['outg_rec_nb']==outg_rec_nb_i]['serialnumber'].unique().tolist()
    expect_with_events = [x for x in SNs_in_outg_rec_nb_i
                          if x in SNs_in_end_events_i]

    expect_no_events = [x for x in SNs_in_outg_rec_nb_i
                        if x not in SNs_in_end_events_i]  
    
    # Should be True
    all_expected_found = all([x in sns_with_end_events['serialnumber'].tolist() for x in expect_with_events])
    
    # Should be False
    any_unexpected_found = any([x in sns_with_end_events['serialnumber'].tolist() for x in expect_no_events])
    
    if not all_expected_found or any_unexpected_found:
        print(outg_rec_nb_i)
        outg_rec_nbs_w_missing_SNs.append(outg_rec_nb_i)
        if not all_expected_found:
            outg_rec_nbs_wo_all_found.append(outg_rec_nb_i)
        if any_unexpected_found:
            outg_rec_nbs_w_unexpected.append(outg_rec_nb_i)
    
#     assert(all_expected_found)
#     assert(not any_unexpected_found)

In [ ]:
t_max

In [ ]:
any_unexpected_found

In [ ]:
[x for x in expect_no_events if x in sns_with_end_events['serialnumber'].tolist()]

In [ ]:
n_skipped=0
outg_rec_nbs_w_missing_prems = []
outg_rec_nbs_wo_all_found = []
outg_rec_nbs_w_unexpected = []
print(len(outg_rec_nbs_w_meters_wo_events))
for i,outg_rec_nb_i in enumerate(outg_rec_nbs_w_meters_wo_events):
# for i,outg_rec_nb_i in enumerate(['10413222']):
    if i%100==0:
        print(i)
    # NOT EXACTLY RIGHT!!!!!
    t_max = outg_dt_off_df.loc[outg_rec_nb_i]['DT_OFF_TS_FULL']
    t_min = t_max-datetime.timedelta(days=30)
    
    if t_max.year>2022 or t_min.year<2018:
        n_skipped += 1
        continue

    prem_nbs_with_end_events = get_SNs_with_end_events_from_local_files(
        date_0=t_min, 
        date_1=t_max, 
        files_dir_base=r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\prem_nbs_with_end_events'
    )

    # For each premise number in the outage (from SNs_in_outgs.loc[outg_rec_nb_i]['premise_nbs'])
    #   if premise number contained in end_events_df_i[end_events_df_i['outg_rec_nb']==outg_rec_nb_i], it is expected to have an event
    #     as determined now by prem_nbs_with_end_events
    #   Otherwise, not expected to have an event in prem_nbs_with_end_events
    prems_in_outg_i_from_DOVS = SNs_in_outgs.loc[outg_rec_nb_i]['premise_nbs']
    prems_in_outg_i_from_end_events = end_events_df_i[end_events_df_i['outg_rec_nb']==outg_rec_nb_i]['aep_premise_nb'].unique().tolist()
    
    if len(set(prems_in_outg_i_from_end_events).difference(set(prems_in_outg_i_from_DOVS)))>0:
        print('I dont understand')
    
    expect_with_events = [x for x in prems_in_outg_i_from_DOVS
                          if x in prems_in_outg_i_from_end_events]

    expect_no_events = [x for x in prems_in_outg_i_from_DOVS
                        if x not in prems_in_outg_i_from_end_events]  
    
    # Should be True
    all_expected_found = all([x in prem_nbs_with_end_events for x in expect_with_events])
    
    # Should be False
    any_unexpected_found = any([x in prem_nbs_with_end_events for x in expect_no_events])
    
    if not all_expected_found:
        print(f'not all_expected_found: {outg_rec_nb_i}')
        
    if any_unexpected_found:
        print(f'any_unexpected_found: {outg_rec_nb_i}')
        prem_nbs_with_end_events = AMIEndEvents.get_end_event_distinct_fields(
            date_0=t_min, 
            date_1=t_max, 
            fields=['aep_premise_nb'], 
            are_datetime=True, 
            addtnl_build_sql_function_kwargs = dict(opco='oh')
        )
        prem_nbs_with_end_events=prem_nbs_with_end_events['aep_premise_nb'].tolist()
        any_unexpected_found = any([x in prem_nbs_with_end_events for x in expect_no_events])
        print(f'Is it really true?: {any_unexpected_found}')
    
    if not all_expected_found or any_unexpected_found:
        #print(outg_rec_nb_i)
        outg_rec_nbs_w_missing_prems.append(outg_rec_nb_i)
        if not all_expected_found:
            outg_rec_nbs_wo_all_found.append(outg_rec_nb_i)
        if any_unexpected_found:
            outg_rec_nbs_w_unexpected.append(outg_rec_nb_i)
    
#     assert(all_expected_found)
#     assert(not any_unexpected_found)

In [ ]:
print('Hello')

In [ ]:
any_unexpected_found

In [ ]:
set(prems_in_outg_i_from_end_events).difference(set(prem_nbs_with_end_events['aep_premise_nb'].tolist()))

In [ ]:
set(prems_in_outg_i_from_DOVS).difference(set(prem_nbs_with_end_events['aep_premise_nb'].tolist()))

In [ ]:
set(prems_in_outg_i_from_DOVS).difference(set(prems_in_outg_i_from_end_events))

In [ ]:
'102398033' in prem_nbs_with_end_events['aep_premise_nb'].tolist()

In [ ]:
# Case example: prem_nb = '102398033'
# Apparently, no events for this premise number were found
# HOWEVER, the outage database claims it to have been affected, and the prem_nbs_with_end_events
#   claims it has an event!
print('102398033' in prems_in_outg_i_from_end_events)
print('102398033' in prems_in_outg_i_from_DOVS)
print('102398033' in prem_nbs_with_end_events['aep_premise_nb'].tolist())

In [ ]:
end_events_df_i[end_events_df_i['aep_premise_nb']=='102398033']

In [ ]:
ede_i = AMIEndEvents(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    contstruct_df_args=None, 
    init_df_in_constructor=True, 
    build_sql_function_kwargs=dict(
        cols_of_interest=TableInfos.AMIEndEvents_TI.std_columns_of_interest, 
        premise_nb='102398033', 
        date_range=[t_min, t_max]
    )
)

In [ ]:
ede_i.df

In [ ]:
t_min

In [ ]:
t_max

In [ ]:
rcpo_df_OG.columns.get_level_values(1)

In [ ]:
rcpo_df_OG.loc['10248878'][('counts', '_SNs')]

In [ ]:
outg_rec_nbs_wo_all_found[0]

In [ ]:
print(len(outg_rec_nbs_w_missing_prems))
print(len(outg_rec_nbs_wo_all_found))
print(len(outg_rec_nbs_w_unexpected))

In [ ]:
# outg_rec_nb_i=outg_rec_nbs_w_unexpected[0]
outg_rec_nb_i = '10253808'

# NOT EXACTLY RIGHT!!!!!
t_max = outg_dt_off_df.loc[outg_rec_nb_i]['DT_OFF_TS_FULL']
t_min = t_max-datetime.timedelta(days=30)

#     sns_with_end_events = AMIEndEvents.get_end_event_distinct_fields(
#         date_0=t_min, 
#         date_1=t_max, 
#         fields=['serialnumber'], 
#         addtnl_build_sql_function_kwargs = dict(opco='oh')
#     )

prem_nbs_with_end_events = AMIEndEvents.get_end_event_distinct_fields(
    date_0=t_min, 
    date_1=t_max, 
    fields=['aep_premise_nb'], 
    addtnl_build_sql_function_kwargs = dict(opco='oh')
)


prems_in_outg_i_from_DOVS = SNs_in_outgs.loc[outg_rec_nb_i]['premise_nbs']
prems_in_outg_i_from_end_events = end_events_df_i[end_events_df_i['outg_rec_nb']==outg_rec_nb_i]['aep_premise_nb'].unique().tolist()

if len(set(prems_in_outg_i_from_end_events).difference(set(prems_in_outg_i_from_DOVS)))>0:
    print('I dont understand')

expect_with_events = [x for x in prems_in_outg_i_from_DOVS
                      if x in prems_in_outg_i_from_end_events]

expect_with_events2 = prems_in_outg_i_from_end_events

expect_no_events = [x for x in prems_in_outg_i_from_DOVS
                    if x not in prems_in_outg_i_from_end_events]  

expect_no_events2 = set(prems_in_outg_i_from_DOVS).difference(set(prems_in_outg_i_from_end_events))

# Should be True
all_expected_found = all([x in prem_nbs_with_end_events['aep_premise_nb'].tolist() for x in expect_with_events])
all_expected_found2 = [x for x in expect_with_events if x in prem_nbs_with_end_events['aep_premise_nb'].tolist()]

# Should be False
any_unexpected_found = any([x in prem_nbs_with_end_events['aep_premise_nb'].tolist() for x in expect_no_events])
any_unexpected_found2 = [x for x in expect_no_events if x in prem_nbs_with_end_events['aep_premise_nb'].tolist()]

In [ ]:
print(len(expect_no_events))
print(len(expect_no_events2))

In [ ]:
len(expect_with_events)

In [ ]:
expect_with_events[0]

In [ ]:
all_expected_found

In [ ]:
any_unexpected_found

In [ ]:
any_unexpected_found2

In [ ]:
outg_rec_nb = '10253808'
end_events_df_i[end_events_df_i['outg_rec_nb']==outg_rec_nb]['aep_premise_nb'].tolist()

In [ ]:
all([x in prem_nbs_with_end_events['aep_premise_nb'].tolist() 
 for x in end_events_df_i[end_events_df_i['outg_rec_nb']==outg_rec_nb]['aep_premise_nb'].tolist()])

In [ ]:
prem_nbs_with_end_events.sort_values(by=['aep_premise_nb'])

In [ ]:
is_no_outg = True
paths = Utilities.find_all_paths(base_dir=files_dir_no_outg, glob_pattern=file_path_glob)
i_beg = 0
i_end = 1000
# i_end=len(paths)
cols_and_types_to_convert_dict=None
to_numeric_errors='coerce'
assert_all_cols_equal=True
outg_rec_nb_col='trsf_pole_nb'
addtnl_dropna_subset_cols=None

days_min_outg_td_window=0
days_max_outg_td_window=30

min_outg_td_window=datetime.timedelta(days=days_min_outg_td_window)
max_outg_td_window=datetime.timedelta(days=days_max_outg_td_window)

In [ ]:
if is_no_outg:
    # If min_outg_td_window or max_outg_td_window is not None, enforce time restrictions around outages
    if min_outg_td_window is not None or max_outg_td_window is not None:
        end_events_df_i=Utilities_dt.strip_tz_info_and_convert_to_dt(
            df=end_events_df_i, 
            time_col='valuesinterval', 
            placement_col='valuesinterval_local', 
            run_quick=True, 
            n_strip=6, 
            inplace=False
        )

        no_outg_time_infos_df = MECPOAn.get_bsln_time_interval_infos_df_from_summary_files(
            summary_paths=[AMIEndEvents.find_summary_file_from_csv(x) for x in paths[i_beg:i_end]], 
            output_prem_nbs_col='prem_nbs', 
            output_t_min_col='t_min', 
            output_t_max_col='DT_OFF_TS_FULL', 
            make_prem_nbs_idx=True
        )
        assert(end_events_df_i['aep_premise_nb'].dtype==no_outg_time_infos_df.index.dtype)
        no_outg_time_infos_series = no_outg_time_infos_df['DT_OFF_TS_FULL']    

        end_events_df_i = AMIEndEvents.enforce_end_events_within_interval_of_outage(
            end_events_df=end_events_df_i, 
            outg_times_series=no_outg_time_infos_series, 
            min_timedelta=min_outg_td_window, 
            max_timedelta=max_outg_td_window, 
            outg_rec_nb_col = 'aep_premise_nb', 
            datetime_col='valuesinterval_local'
        )

In [ ]:
end_events_df_i.shape

In [ ]:
end_events_df_i.head()

In [ ]:
print(end_events_df_i['trsf_pole_nb'].nunique())

In [ ]:
end_events_df_ij = end_events_df_i[end_events_df_i['outg_rec_nb']==end_events_df_i['outg_rec_nb'].unique()[0]]
end_events_df_ij

In [ ]:
assert(end_events_df_ij['outg_rec_nb'].nunique()==1)
end_events_df_ij['outg_rec_nb'].unique()[0]

In [ ]:
outg_dt_off_df.loc[end_events_df_ij['outg_rec_nb'].unique()[0]]['DT_OFF_TS_FULL']

In [ ]:
t_max = outg_dt_off_df.loc[end_events_df_ij['outg_rec_nb'].unique()[0]]['DT_OFF_TS_FULL']
t_min = t_max-datetime.timedelta(days=30)

In [ ]:
sns_with_end_events = AMIEndEvents.get_end_event_distinct_fields(
    date_0=t_min, 
    date_1=t_max, 
    fields=['serialnumber'], 
    addtnl_build_sql_function_kwargs = dict(opco='oh')
)

In [ ]:
[x for x in end_events_df_ij['serialnumber'].unique().tolist() if x in sns_with_end_events['serialnumber'].tolist()]

In [ ]:
no_outg_time_infos_df = MECPOAn.get_bsln_time_interval_infos_df_from_summary_files(
#     summary_paths=[AMIEndEvents.find_summary_file_from_csv(x) for x in paths[i_beg:i_end]], 
    summary_paths=[AMIEndEvents.find_summary_file_from_csv(x) for x in paths], 
    output_prem_nbs_col='prem_nbs', 
    output_t_min_col='t_min', 
    output_t_max_col='DT_OFF_TS_FULL', 
    make_prem_nbs_idx=True
)

In [ ]:
assert(no_outg_time_infos_df['t_min'].nunique()==1)
assert(no_outg_time_infos_df['DT_OFF_TS_FULL'].nunique()==1)

In [ ]:
no_outg_time_infos_df.index.nunique()

In [ ]:
no_outg_time_infos_df.iloc[0]

In [ ]:
no_outg_time_infos_df

In [ ]:
mp_xfmr_poles_from_end_events = MeterPremise(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    contstruct_df_args=None, 
    init_df_in_constructor=True, 
    build_sql_function_kwargs=dict(
        cols_of_interest=TableInfos.MeterPremise_TI.std_columns_of_interest, 
#         trsf_pole_nbs=end_events_df_i['trsf_pole_nb'].unique().tolist()
    )
)

mp_xfmr_poles_from_end_events_df = mp_xfmr_poles_from_end_events.df.copy()
print(mp_xfmr_poles_from_end_events_df.shape)
mp_xfmr_poles_from_end_events_df = mp_xfmr_poles_from_end_events_df[mp_xfmr_poles_from_end_events_df['trsf_pole_nb'].isin(end_events_df_i['trsf_pole_nb'].unique().tolist())]
print(mp_xfmr_poles_from_end_events_df.shape)

In [ ]:
print(mp_xfmr_poles_from_end_events.df.shape)
print(no_outg_time_infos_df.shape)

In [ ]:
# Need to find all meters on each transformer
# Each meter needs to either be in sample of meters without an event, or in collected sample
# If not in either, than sample needs to be collected

In [ ]:
mp_xfmr_poles_from_end_events_df = mp_xfmr_poles_from_end_events.df.copy()

In [ ]:
mp_xfmr_poles_from_end_events_df

In [ ]:
print(mp_xfmr_poles_from_end_events_df['mfr_devc_ser_nbr'].nunique())
print(mp_xfmr_poles_from_end_events_df['prem_nb'].nunique())

In [ ]:
mp_xfmr_poles_from_end_events_df['prem_nb'].dtype==no_outg_time_infos_df.index.dtype

In [ ]:
should_have_times = [x for x in mp_xfmr_poles_from_end_events_df['prem_nb'].tolist() 
                     if x in no_outg_time_infos_df.index.tolist()]
print(len(should_have_times))

In [ ]:
should_not_have_times = [x for x in mp_xfmr_poles_from_end_events_df['prem_nb'].tolist() 
                     if x not in no_outg_time_infos_df.index.tolist()]
print(len(should_not_have_times))

In [ ]:
mp_xfmr_poles_from_end_events_merged_with_no_outg_time_infos_df = pd.merge(
    mp_xfmr_poles_from_end_events_df, 
    no_outg_time_infos_df, left_on='prem_nb', right_index=True, how='left')

In [ ]:
mp_xfmr_poles_from_end_events_merged_with_no_outg_time_infos_df

In [ ]:
mp_xfmr_poles_from_end_events_merged_with_no_outg_time_infos_df[mp_xfmr_poles_from_end_events_merged_with_no_outg_time_infos_df['t_min'].notna()]

In [ ]:
assert(no_outg_time_infos_df['t_min'].nunique()==1)
assert(no_outg_time_infos_df['DT_OFF_TS_FULL'].nunique()==1)
print(pd.to_datetime(no_outg_time_infos_df['t_min'].unique().tolist()[0]))
print(pd.to_datetime(no_outg_time_infos_df['DT_OFF_TS_FULL'].unique().tolist()[0]))

In [ ]:
sns_with_end_events = AMIEndEvents.get_end_event_distinct_fields(
    date_0=pd.to_datetime(no_outg_time_infos_df['t_min'].unique().tolist()[0]), 
    date_1=pd.to_datetime(no_outg_time_infos_df['DT_OFF_TS_FULL'].unique().tolist()[0]), 
    fields=['serialnumber'], 
    addtnl_build_sql_function_kwargs = dict(opco='oh')
)

In [ ]:
sns_with_end_events

In [ ]:
mp_xfmr_poles_from_end_events_df

In [ ]:
idk1 = [x for x in mp_xfmr_poles_from_end_events_df['mfr_devc_ser_nbr'].tolist() 
        if x in sns_with_end_events['serialnumber'].tolist()]

In [ ]:
len(idk1)

In [ ]:
idk2 = [x for x in mp_xfmr_poles_from_end_events_df['mfr_devc_ser_nbr'].tolist() 
        if x not in sns_with_end_events['serialnumber'].tolist()]

In [ ]:
len(idk2)

In [ ]:
end_events_df_i[end_events_df_i['serialnumber'].isin(idk2)]

In [ ]:
# These should have no end events
should_have_no_events = mp_xfmr_poles_from_end_events_merged_with_no_outg_time_infos_df[
    mp_xfmr_poles_from_end_events_merged_with_no_outg_time_infos_df['t_min'].isna()
]['mfr_devc_ser_nbr'].tolist()

# These should have end events
should_have_events = mp_xfmr_poles_from_end_events_merged_with_no_outg_time_infos_df[
    mp_xfmr_poles_from_end_events_merged_with_no_outg_time_infos_df['t_min'].notna()
]['mfr_devc_ser_nbr'].tolist()

In [ ]:
print(mp_xfmr_poles_from_end_events_merged_with_no_outg_time_infos_df.shape)
print(len(should_have_no_events))
print(len(should_have_events))

In [ ]:
sum([x in sns_with_end_events['serialnumber'].tolist() for x in should_have_no_events])

In [ ]:
sum([x in sns_with_end_events['serialnumber'].tolist() for x in should_have_events])

In [3]:
print("WITH MP AS (\n\tSELECT\n\t\tmfr_devc_ser_nbr,\n\t\tlongitude,\n\t\tlatitude,\n\t\tstate_cd,\n\t\tprem_nb,\n\t\tsrvc_pole_nb,\n\t\ttrsf_pole_nb,\n\t\tlatitude_nb,\n\t\tlongitude_nb,\n\t\tcircuit_nb,\n\t\tcircuit_nm,\n\t\tstation_nb,\n\t\tstation_nm,\n\t\txfmr_nb,\n\t\tannual_kwh,\n\t\tannual_max_dmnd,\n\t\tmtr_stat_cd,\n\t\tmtr_stat_cd_desc,\n\t\tdevc_stat_cd,\n\t\tdevc_stat_cd_desc,\n\t\tcurr_acct_cls_cd\n\tFROM default.meter_premise\n) \nSELECT\n\tun_rin.issuertracking_id,\n\tun_rin.serialnumber,\n\tun_rin.enddeviceeventtypeid,\n\tun_rin.valuesinterval,\n\tun_rin.aep_premise_nb,\n\tun_rin.reason,\n\tun_rin.event_type,\n\tun_rin.aep_opco,\n\tun_rin.aep_event_dt,\n\t'10248878' AS OUTG_REC_NB_GPD_FOR_SQL,\n\tMP.*\nFROM meter_events.end_device_event un_rin\n\tLEFT JOIN MP ON un_rin.serialnumber=MP.mfr_devc_ser_nbr\nWHERE un_rin.aep_event_dt BETWEEN '2017-12-02' AND '2018-01-31'\nAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2017-12-02 02:19:00' AND TIMESTAMP '2018-01-31 04:40:00'\nAND   un_rin.aep_premise_nb IN ('107998033','108288033','108598033','102398033','105709033','105188033','106309033','107378033')",)

WITH MP AS (
	SELECT
		mfr_devc_ser_nbr,
		longitude,
		latitude,
		state_cd,
		prem_nb,
		srvc_pole_nb,
		trsf_pole_nb,
		latitude_nb,
		longitude_nb,
		circuit_nb,
		circuit_nm,
		station_nb,
		station_nm,
		xfmr_nb,
		annual_kwh,
		annual_max_dmnd,
		mtr_stat_cd,
		mtr_stat_cd_desc,
		devc_stat_cd,
		devc_stat_cd_desc,
		curr_acct_cls_cd
	FROM default.meter_premise
) 
SELECT
	un_rin.issuertracking_id,
	un_rin.serialnumber,
	un_rin.enddeviceeventtypeid,
	un_rin.valuesinterval,
	un_rin.aep_premise_nb,
	un_rin.reason,
	un_rin.event_type,
	un_rin.aep_opco,
	un_rin.aep_event_dt,
	'10248878' AS OUTG_REC_NB_GPD_FOR_SQL,
	MP.*
FROM meter_events.end_device_event un_rin
	LEFT JOIN MP ON un_rin.serialnumber=MP.mfr_devc_ser_nbr
WHERE un_rin.aep_event_dt BETWEEN '2017-12-02' AND '2018-01-31'
AND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2017-12-02 02:19:00' AND TIMESTAMP '2018-01-31 04:40:

# DEV FOR NO OUTAGE NORMALIZATION

In [ ]:
# rcpo_df_raw = MECPODf.add_outage_SNs_to_rcpo_df(
#     rcpo_df=rcpo_df_raw, 
#     set_outage_nSNs=True, 
#     include_outage_premise_nbs=True    
# )

In [ ]:
rcpo_no_outg_df_raw

In [ ]:
trsf_pole_nbs=rcpo_no_outg_df_raw.index.tolist()[:10]
SNs_and_PNs_for_xfmrs = MeterPremise.get_SNs_for_xfmrs(
    trsf_pole_nbs=trsf_pole_nbs
)

In [ ]:
SNs_and_PNs_for_xfmrs

In [ ]:
# # Build no_outg_time_infos_df, which has prem_nbs indices and t_min, t_max (and possible summary_path) columns
# # This is where the time information for each premise number comes from
# files_dir_no_outg             = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\EndEvents_NoOutg'
# paths_no_outg = Utilities.find_all_paths(base_dir=files_dir_no_outg, glob_pattern=file_path_glob)

# no_outg_time_infos_df = MECPOAn.get_bsln_time_interval_infos_df_from_summary_files(
#     summary_paths=[AMIEndEvents.find_summary_file_from_csv(x) for x in paths_no_outg], 
#     output_prem_nbs_col='prem_nbs', 
#     output_t_min_col='t_min', 
#     output_t_max_col='t_max', 
#     make_prem_nbs_idx=True, 
#     include_summary_paths=True
# )

In [ ]:
# mp_df_curr_hist_no_outgs = build_mp_df_curr_hist_for_xfmrs(trsf_pole_nbs=rcpo_no_outg_df_raw.index.tolist()[:10])

In [ ]:
df_mp_curr = mp_df_curr_hist_no_outgs['mp_df_curr']
df_mp_hist = mp_df_curr_hist_no_outgs['mp_df_hist']

In [ ]:
active_SNs_df = get_active_SNs_for_xfmrs_OLD(
    trsf_pole_nbs=rcpo_no_outg_df_raw.index.tolist()[:10], 
    df_mp_curr=df_mp_curr, 
    df_mp_hist=df_mp_hist,
    no_outg_time_infos_df=no_outg_time_infos_df
)

In [ ]:
active_SNs_df

In [ ]:
active_SNs_df

In [ ]:
rcpo_no_outg_df_raw

In [ ]:
test_rcpo_df = rcpo_no_outg_df_raw.iloc[:10].copy()

In [ ]:
test_rcpo_df = add_xfmr_active_SNs_to_rcpo_df(
    rcpo_df=test_rcpo_df, 
    set_xfmr_nSNs=True, 
    include_active_xfmr_PNs=True,
    df_mp_curr=df_mp_curr,
    df_mp_hist=df_mp_hist, 
    no_outg_time_infos_df=no_outg_time_infos_df
)

In [ ]:
test_rcpo_df

In [ ]:

assert('_xfmr_nSNs' in test_rcpo_df.columns)
assert('_xfmr_SNs' in test_rcpo_df.columns)
test_rcpo_df_2 = build_rcpo_df_norm_by_outg_nSNs_NEW(
    rcpo_df_raw=test_rcpo_df, 
    outg_nSNs_col='_xfmr_nSNs', 
    outg_SNs_col='_xfmr_SNs', 
    other_SNs_col_tags_to_ignore=['_SNs', '_nSNs', '_prem_nbs', '_nprem_nbs', '_xfmr_PNs', '_xfmr_nPNs'], 
    drop_outg_nSNs_eq_0=True, 
    new_level_0_val='counts_norm_by_xfmr_nSNs', 
    remove_SNs_cols=False, 
    df_mp_curr=df_mp_curr,
    df_mp_hist=df_mp_hist    
)

In [ ]:
test_rcpo_df.head()

In [ ]:
test_rcpo_df_2.head()

In [ ]:
# def get_existing_full_rcpo_col(
#     col_str, 
#     rcpo_df, 
#     max_nlevels=2, 
#     assert_single_unique_value_for_lower_levels=True
# ):
#     r"""
#     Returns the full column name for col_str.
#     If the columns of rcpo_df are normal Index items, then this simply returns col_str.
#     If the columns of rcpo_df are MultiIndex objects, this will return (level_0, col_str)
    
#     Designed for the case where rcpo_df has at most two levels, and level 0 has a single unique value.
#     Will work with DFs having higher order column levels, and will work if lower levels have more than
#       one unique value per level.
#     HOWEVER!!!!!
#       If the lower levels have more than one unique value per level, there can ONLY BE ONE INSTANCE of col_str
#       in the highest level.  Otherwise, there would be degeneracy, and it would be uncertain which column was desired
#         e.g., consider a DF with columns ('counts', 'Reason_1') and ('counts_norm', 'Reason_1').
#                 If col_str='Reason_1', there would be no way of knowing which of these to return!
#               However, if instead we had a DF with columns ('counts', 'Reason_1') and ('counts_norm', 'Reason_2'), then
#                 there is no ambiguity, and there is also no issue with level 0 having more than one unique value!
#     """
#     #-------------------------
#     n_levels = rcpo_df.columns.nlevels
#     assert(n_levels<=max_nlevels)
#     #-------------------------
#     # If only one level, simply return col_str (assuming it is found in rcpo_df.columns!)
#     if n_levels==1:
#         assert(col_str in rcpo_df.columns)
#         return col_str
#     #-------------------------
#     if assert_single_unique_value_for_lower_levels:
#         for i_level in range(n_levels-1):
#             assert(rcpo_df.columns.get_level_values(i_level).nunique()==1)
#     #-------------------------
#     highest_level_col_values = rcpo_df.columns.get_level_values(n_levels-1).tolist()
#     assert(col_str in highest_level_col_values)
#     #-----
#     found_idxs = Utilities.find_tagged_idxs_in_list(highest_level_col_values, [col_str+'_EXACTMATCH'])
#     # There can only be one found, otherwise there is ambiguity!
#     assert(len(found_idxs)==1)
#     return_col = rcpo_df.columns[found_idxs[0]]
#     #-------------------------
#     return return_col

# def build_full_rcpo_col(
#     col_str, 
#     rcpo_df, 
#     max_nlevels=2
# ):
#     r"""
#     If the columns of rcpo_df are MultiIndex, this will essentially build a tuple containing the lower level column values
#       and ending with col_str.
#     This is different from get_existing_full_rcpo_col in that col_str does not need to already exist in rcpo_df.
#     However, for this to make sense, when the columns of rcpo_df are MultiIndex, all lower level values
#       must contain only a single unique value (otherwise, there is ambiguity in which value should be chosen!)
#     """
#     #-------------------------
#     n_levels = rcpo_df.columns.nlevels
#     assert(n_levels<=max_nlevels)
#     #-------------------------
#     return_col = []
#     for i_level in range(n_levels-1):
#         assert(rcpo_df.columns.get_level_values(i_level).nunique()==1)
#         return_col.append(rcpo_df.columns.get_level_values(i_level).unique().tolist()[0])
#     return_col.append(col_str)
#     #-------------------------
#     assert(len(return_col)==n_levels)
#     return_col = tuple(return_col)
#     return return_col

In [ ]:
# def build_rcpo_df_norm_by_list_counts(
#     rcpo_df_raw, 
#     n_counts_col='_outg_nSNs', 
#     list_col='_outg_SNs', 
#     add_list_col_to_rcpo_df_func=None, 
#     add_list_col_to_rcpo_df_kwargs={}, 
#     other_col_tags_to_ignore=['_SNs', '_nSNs', '_prem_nbs', '_nprem_nbs'], 
#     drop_n_counts_eq_0=True, 
#     new_level_0_val='counts_norm_by_outg_nSNs', 
#     remove_ignored_cols=False
# ):
#     r"""
#     Build rcpo_df normalized by the number of elements in list_col (which are stored in n_counts_col).
    
#     If n_counts_col exists in rcpo_df_raw, the normalized DF is simply formed by dividing the columns to be
#       normalized (see notes below on other_col_tags_to_ignore) by n_counts_col.
#     If n_counts_col does not exist:
#       If list_col exists in rcpo_df_raw, then n_counts_col is built by simply taking the length of each list
#         element in list_col.  The normalization then proceeds simply as described above.
#       If list_col does not exist, it will be built using the function add_list_col_to_rcpo_df_func together with the
#         function arguments rcpo_df=rcpo_df_norm, **add_list_col_to_rcpo_df_kwargs
    
#     drop_n_counts_eq_0:
#       It is possible for the number of counts to be zero.
#       Dividing by zero will make all counts for such an entry equal to NaN or inf.
#       When drop_n_counts_eq_0 is True, such entries will be removed.

#     NOTE: list_col and n_counts_col should both be strings, not tuples.
#           If column is MultiIndex, the level_0 value will be handled below.
#     """
#     #-------------------------
#     rcpo_df_norm = rcpo_df_raw.copy()
#     #-------------------------
#     assert(rcpo_df_norm.columns.nlevels<=2)
#     if rcpo_df_norm.columns.nlevels==1:
#         reason_level_idx=0
#         col_tags_to_ignore = other_col_tags_to_ignore + [n_counts_col+'_EXACTMATCH', list_col+'_EXACTMATCH']
#     else:
#         # Should only be raw df, so if n_col_levels==2, should only be one unique value in level 0
#         assert(rcpo_df_norm.columns.get_level_values(0).nunique()==1)
#         level_0_val = rcpo_df_norm.columns.get_level_values(0).unique().tolist()[0]
#         #-----
#         if isinstance(list_col, str):
#             list_col = MECPODf.build_full_rcpo_col(col_str=list_col, rcpo_df=rcpo_df_norm, max_nlevels=2)
#         assert(isinstance(list_col,tuple) and len(list_col)==2 and list_col[0]==level_0_val)
#         #-----
#         if isinstance(n_counts_col, str):
#             n_counts_col = MECPODf.build_full_rcpo_col(col_str=n_counts_col, rcpo_df=rcpo_df_norm, max_nlevels=2)
#         assert(isinstance(n_counts_col,tuple) and len(n_counts_col)==2 and n_counts_col[0]==level_0_val)
#         #---------------
#         reason_level_idx=1
#         col_tags_to_ignore = other_col_tags_to_ignore + [n_counts_col[1]+'_EXACTMATCH', list_col[1]+'_EXACTMATCH']
#     #-------------------------
#     # If n_counts_col, which will be used to normalize, is not in rcpo_df_norm, build it
#     if n_counts_col not in rcpo_df_norm.columns:
#         # If for some reason list_col exists (although n_counts_col doesn't), simply call 
#         # MECPODf.set_nSNs_from_SNs_in_rcpo_df instead of running full-blown add_list_col_to_rcpo_df_func
#         if list_col in rcpo_df_norm.columns:
#             rcpo_df_norm = MECPODf.set_nSNs_from_SNs_in_rcpo_df(
#                 rcpo_df_wide=rcpo_df_norm, 
#                 SNs_col=list_col, 
#                 nSNs_col=n_counts_col
#             )
#         else:
#             # More typical case, where both n_counts_col and list_col are absent from rcpo_df_norm
#             assert(add_list_col_to_rcpo_df_func is not None)
#             rcpo_df_norm = add_list_col_to_rcpo_df_func(
#                 rcpo_df=rcpo_df_norm, 
#                 **add_list_col_to_rcpo_df_kwargs
#             )
#     #-------------------------
#     # Find the columns to normalize as those not in col_tags_to_ignore
#     cols_to_norm_idxs = Utilities.find_untagged_idxs_in_list(
#         lst=rcpo_df_norm.columns.get_level_values(reason_level_idx).tolist(), 
#         tags=col_tags_to_ignore)
#     cols_to_norm = rcpo_df_norm.columns[cols_to_norm_idxs]
#     norm_col = n_counts_col
#     #-----
#     # Casting columns as np.float64 allows the operation of dividing by zero (should result in NaN (or inf in some instances))
#     # This will happen when no SNs are found for an outage 
#     #   NOTE: premise numbers are always found, but the meter_premise database does not always contain the premise number
#     rcpo_df_norm = Utilities_df.convert_col_types(
#         df=rcpo_df_norm, 
#         cols_and_types_dict = {x:np.float64 for x in cols_to_norm.tolist()+[norm_col]}, 
#         to_numeric_errors='coerce', 
#         inplace=True
#     )
#     #-----
#     rcpo_df_norm[cols_to_norm] = rcpo_df_norm[cols_to_norm].divide(rcpo_df_norm[norm_col], axis=0)
#     #-------------------------
#     if drop_n_counts_eq_0:
#         rcpo_df_norm = rcpo_df_norm.drop(index=rcpo_df_norm[rcpo_df_norm[n_counts_col]==0].index)
#     #-------------------------
#     if rcpo_df_norm.columns.nlevels==2:
#         rcpo_df_norm.columns = rcpo_df_norm.columns.set_levels([new_level_0_val], level=0)
#     #-------------------------
#     if remove_ignored_cols:
#         rcpo_df_norm = MECPODf.remove_SNs_cols_from_rcpo_df(
#             rcpo_df=rcpo_df_norm, 
#             SNs_tags=col_tags_to_ignore, 
#             is_long=False
#         )
#     #-------------------------
#     return rcpo_df_norm

In [ ]:
# def build_rcpo_df_norm_by_outg_nSNs_2(
#     rcpo_df_raw, 
#     outg_nSNs_col='_outg_nSNs', 
#     outg_SNs_col='_outg_SNs', 
#     other_SNs_col_tags_to_ignore=['_SNs', '_nSNs', '_prem_nbs', '_nprem_nbs'], 
#     drop_outg_nSNs_eq_0=True, 
#     new_level_0_val='counts_norm_by_outg_nSNs', 
#     remove_SNs_cols=False
# ):
#     r"""
#     Build rcpo_df normalized by the number of serial numbers in each outage

#     drop_outg_nSNs_eq_0:
#       It is possible for the number of serial numbers in an outage to be zero!
#       Premise numbers are always found, but the meter_premise database does not always 
#         contain the premise numbers.
#       Dividing by zero will make all counts for such an entry equal to NaN or inf.
#       When drop_outg_nSNs_eq_0 is True, such entries will be removed.

#     NOTE: outg_SNs_col and outg_nSNs_col should both be strings, not tuples.
#           If column is multiindex, the level_0 value will be handled below.
#     """
#     #-------------------------
#     n_counts_col = outg_nSNs_col
#     list_col = outg_SNs_col
#     #-------------------------
#     # NOTE: MECPODf.add_outage_SNs_to_rcpo_df expects outg_SNs_col and outg_nSNs_col to be strings, not tuples
#     #       as it handles the level 0 values if they exist.  So, if tuples, use only highest level values (i.e., level 1)
#     assert(Utilities.is_object_one_of_types(list_col, [str, tuple]))
#     assert(Utilities.is_object_one_of_types(n_counts_col, [str, tuple]))
#     #-----
#     add_list_col_to_rcpo_df_func = MECPODf.add_outage_SNs_to_rcpo_df
#     add_list_col_to_rcpo_df_kwargs = dict(
#         set_outage_nSNs=True, 
#         include_outage_premise_nbs=False, 
#         outg_SNs_col =(outg_SNs_col  if isinstance(outg_SNs_col, str)  else outg_SNs_col[1]), 
#         outg_nSNs_col=(outg_nSNs_col if isinstance(outg_nSNs_col, str) else outg_nSNs_col[1])
#     )
#     #-------------------------
#     other_col_tags_to_ignore = other_SNs_col_tags_to_ignore
#     drop_n_counts_eq_0 = drop_outg_nSNs_eq_0
#     new_level_0_val = new_level_0_val
#     remove_ignored_cols = remove_SNs_cols
#     #-------------------------
#     return MECPODf.build_rcpo_df_norm_by_list_counts(
#         rcpo_df_raw=rcpo_df_raw, 
#         n_counts_col=n_counts_col, 
#         list_col=list_col, 
#         add_list_col_to_rcpo_df_func=add_list_col_to_rcpo_df_func, 
#         add_list_col_to_rcpo_df_kwargs=add_list_col_to_rcpo_df_kwargs, 
#         other_col_tags_to_ignore=other_col_tags_to_ignore, 
#         drop_n_counts_eq_0=drop_n_counts_eq_0, 
#         new_level_0_val=new_level_0_val, 
#         remove_ignored_cols=remove_ignored_cols
#     )

In [ ]:
# def build_rcpo_df_norm_by_outg_active_nSNs(
#     rcpo_df_raw, 
#     outg_nSNs_col='_outg_nSNs', 
#     outg_SNs_col='_outg_SNs', 
#     other_SNs_col_tags_to_ignore=['_SNs', '_nSNs', '_prem_nbs', '_nprem_nbs'], 
#     drop_outg_nSNs_eq_0=True, 
#     new_level_0_val='counts_norm_by_outg_nSNs', 
#     remove_SNs_cols=False, 
#     df_mp_curr=None,
#     df_mp_hist=None
# ):
#     r"""
#     Build rcpo_df normalized by the number of serial numbers in each outage

#     drop_outg_nSNs_eq_0:
#       It is possible for the number of serial numbers in an outage to be zero!
#       Premise numbers are always found, but the meter_premise database does not always 
#         contain the premise numbers.
#       Dividing by zero will make all counts for such an entry equal to NaN or inf.
#       When drop_outg_nSNs_eq_0 is True, such entries will be removed.

#     NOTE: outg_SNs_col and outg_nSNs_col should both be strings, not tuples.
#           If column is multiindex, the level_0 value will be handled below.
#     """
#     #-------------------------
#     n_counts_col = outg_nSNs_col
#     list_col = outg_SNs_col
#     #-------------------------
#     # NOTE: MECPODf.add_outage_SNs_to_rcpo_df expects outg_SNs_col and outg_nSNs_col to be strings, not tuples
#     #       as it handles the level 0 values if they exist.  So, if tuples, use only highest level values (i.e., level 1)
#     assert(Utilities.is_object_one_of_types(list_col, [str, tuple]))
#     assert(Utilities.is_object_one_of_types(n_counts_col, [str, tuple]))
#     #-----
#     add_list_col_to_rcpo_df_func = MECPODf.add_outage_active_SNs_to_rcpo_df
#     add_list_col_to_rcpo_df_kwargs = dict(
#         set_outage_nSNs=True, 
#         include_outage_premise_nbs=False, 
#         outg_SNs_col =(outg_SNs_col  if isinstance(outg_SNs_col, str)  else outg_SNs_col[1]), 
#         outg_nSNs_col=(outg_nSNs_col if isinstance(outg_nSNs_col, str) else outg_nSNs_col[1]), 
#         df_mp_curr=df_mp_curr, 
#         df_mp_hist=df_mp_hist
#     )
#     #-------------------------
#     other_col_tags_to_ignore = other_SNs_col_tags_to_ignore
#     drop_n_counts_eq_0 = drop_outg_nSNs_eq_0
#     new_level_0_val = new_level_0_val
#     remove_ignored_cols = remove_SNs_cols
#     #-------------------------
#     return MECPODf.build_rcpo_df_norm_by_list_counts(
#         rcpo_df_raw=rcpo_df_raw, 
#         n_counts_col=n_counts_col, 
#         list_col=list_col, 
#         add_list_col_to_rcpo_df_func=add_list_col_to_rcpo_df_func, 
#         add_list_col_to_rcpo_df_kwargs=add_list_col_to_rcpo_df_kwargs, 
#         other_col_tags_to_ignore=other_col_tags_to_ignore, 
#         drop_n_counts_eq_0=drop_n_counts_eq_0, 
#         new_level_0_val=new_level_0_val, 
#         remove_ignored_cols=remove_ignored_cols
#     )

In [ ]:
# def build_rcpo_df_norm_by_prim_nSNs_2(
#     rcpo_df_raw, 
#     direct_SNs_in_outgs_df=None, 
#     outg_rec_nb_col='index', 
#     prim_nSNs_col='_prim_nSNs', 
#     prim_SNs_col='_prim_SNs', 
#     other_SNs_col_tags_to_ignore=['_SNs', '_nSNs', '_prem_nbs', '_nprem_nbs'], 
#     drop_prim_nSNs_eq_0=True, 
#     new_level_0_val='counts_norm_by_prim_nSNs', 
#     remove_SNs_cols=False, 
#     build_direct_SNs_in_outgs_df_kwargs={}
# ):
#     r"""
#     Build rcpo_df normalized by the number of primary serial numbers in each outage

#     drop_prim_nSNs_eq_0:
#       It is possible for the number of serial numbers in an outage to be zero!
#       Premise numbers are always found, but the meter_premise database does not always 
#         contain the premise numbers.
#       Dividing by zero will make all counts for such an entry equal to NaN or inf.
#       When drop_prim_nSNs_eq_0 is True, such entries will be removed.

#     NOTE: prim_SNs_col and prim_nSNs_col should both be strings, not tuples.
#           If column is multiindex, the level_0 value will be handled below.
#     """
#     #-------------------------
#     n_counts_col = prim_nSNs_col
#     list_col = prim_SNs_col
#     #-------------------------
#     add_list_col_to_rcpo_df_func = MECPODf.add_prim_SNs_to_rcpo_df
#     add_list_col_to_rcpo_df_kwargs = dict(
#         direct_SNs_in_outgs_df=direct_SNs_in_outgs_df, 
#         outg_rec_nb_col=outg_rec_nb_col, 
#         prim_SNs_col='direct_serial_numbers', 
#         set_prim_nSNs=True, 
#         sort_SNs=True, 
#         build_direct_SNs_in_outgs_df_kwargs=build_direct_SNs_in_outgs_df_kwargs, 
#         placement_prim_SNs_col=prim_SNs_col, 
#         placement_prim_nSNs_col=prim_nSNs_col
#     )
#     #-------------------------
#     other_col_tags_to_ignore = other_SNs_col_tags_to_ignore
#     drop_n_counts_eq_0 = drop_prim_nSNs_eq_0
#     new_level_0_val = new_level_0_val
#     remove_ignored_cols = remove_SNs_cols
#     #-------------------------
#     return MECPODf.build_rcpo_df_norm_by_list_counts(
#         rcpo_df_raw=rcpo_df_raw, 
#         n_counts_col=n_counts_col, 
#         list_col=list_col, 
#         add_list_col_to_rcpo_df_func=add_list_col_to_rcpo_df_func, 
#         add_list_col_to_rcpo_df_kwargs=add_list_col_to_rcpo_df_kwargs, 
#         other_col_tags_to_ignore=other_col_tags_to_ignore, 
#         drop_n_counts_eq_0=drop_n_counts_eq_0, 
#         new_level_0_val=new_level_0_val, 
#         remove_ignored_cols=remove_ignored_cols
#     )

In [ ]:
# def build_rcpo_df_norm_by_prim_active_nSNs(
#     rcpo_df_raw, 
#     direct_SNs_in_outgs_df=None, 
#     outg_rec_nb_col='index', 
#     prim_nSNs_col='_prim_nSNs', 
#     prim_SNs_col='_prim_SNs', 
#     other_SNs_col_tags_to_ignore=['_SNs', '_nSNs', '_prem_nbs', '_nprem_nbs'], 
#     drop_prim_nSNs_eq_0=True, 
#     new_level_0_val='counts_norm_by_prim_nSNs', 
#     remove_SNs_cols=False, 
#     build_direct_SNs_in_outgs_df_kwargs={}, 
#     df_mp_curr=None,
#     df_mp_hist=None
# ):
#     r"""
#     Build rcpo_df normalized by the number of primary serial numbers in each outage

#     drop_prim_nSNs_eq_0:
#       It is possible for the number of serial numbers in an outage to be zero!
#       Premise numbers are always found, but the meter_premise database does not always 
#         contain the premise numbers.
#       Dividing by zero will make all counts for such an entry equal to NaN or inf.
#       When drop_prim_nSNs_eq_0 is True, such entries will be removed.

#     NOTE: prim_SNs_col and prim_nSNs_col should both be strings, not tuples.
#           If column is multiindex, the level_0 value will be handled below.
#     """
#     #-------------------------
#     n_counts_col = prim_nSNs_col
#     list_col = prim_SNs_col
#     #-------------------------
#     add_list_col_to_rcpo_df_func = MECPODf.add_active_prim_SNs_to_rcpo_df
#     add_list_col_to_rcpo_df_kwargs = dict(
#         direct_SNs_in_outgs_df=direct_SNs_in_outgs_df, 
#         outg_rec_nb_col=outg_rec_nb_col, 
#         prim_SNs_col='direct_serial_numbers', 
#         set_prim_nSNs=True, 
#         sort_SNs=True, 
#         build_direct_SNs_in_outgs_df_kwargs=build_direct_SNs_in_outgs_df_kwargs, 
#         placement_prim_SNs_col=prim_SNs_col, 
#         placement_prim_nSNs_col=prim_nSNs_col, 
#         df_mp_curr=df_mp_curr, 
#         df_mp_hist=df_mp_hist
#     )
#     #-------------------------
#     other_col_tags_to_ignore = other_SNs_col_tags_to_ignore
#     drop_n_counts_eq_0 = drop_prim_nSNs_eq_0
#     new_level_0_val = new_level_0_val
#     remove_ignored_cols = remove_SNs_cols
#     #-------------------------
#     return MECPODf.build_rcpo_df_norm_by_list_counts(
#         rcpo_df_raw=rcpo_df_raw, 
#         n_counts_col=n_counts_col, 
#         list_col=list_col, 
#         add_list_col_to_rcpo_df_func=add_list_col_to_rcpo_df_func, 
#         add_list_col_to_rcpo_df_kwargs=add_list_col_to_rcpo_df_kwargs, 
#         other_col_tags_to_ignore=other_col_tags_to_ignore, 
#         drop_n_counts_eq_0=drop_n_counts_eq_0, 
#         new_level_0_val=new_level_0_val, 
#         remove_ignored_cols=remove_ignored_cols
#     )

In [ ]:
df = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\rcpo_dfs_NEW\outg_td_window_1_to_5_days\rcpo_df_OG.pkl')

In [ ]:
df = df.iloc[:100].copy()

In [ ]:
rcpo_df_raw_1  = MECPODf.project_level_0_columns_from_rcpo_wide(df, 'counts', droplevel=False)
rcpo_df_raw_2  = MECPODf.project_level_0_columns_from_rcpo_wide(df, 'counts', droplevel=True)

In [ ]:
start = time.time()
rcpo_df_raw_1 = MECPODf.build_rcpo_df_norm_by_outg_nSNs(rcpo_df_raw_1)
print(time.time()-start)

In [ ]:
start = time.time()
rcpo_df_raw_2 = build_rcpo_df_norm_by_outg_nSNs_2(rcpo_df_raw_2)
print(time.time()-start)

In [ ]:
rcpo_df_raw_1.droplevel(0, axis=1).equals(rcpo_df_raw_2)

In [ ]:
start = time.time()
rcpo_df_raw_1 = MECPODf.build_rcpo_df_norm_by_prim_nSNs(rcpo_df_raw_1)
print(time.time()-start)

In [ ]:
start = time.time()
rcpo_df_raw_2 = build_rcpo_df_norm_by_prim_nSNs_2(rcpo_df_raw_2)
print(time.time()-start)

In [ ]:
rcpo_df_raw_1.droplevel(0, axis=1).equals(rcpo_df_raw_2)